 **模式识别课程设计期末报告**
 
​																																								                                                        22920182204174  郭立敏



​																								    **目录**

​														**一、问题引入、idea可行性及现实意义分析**

​																		**1.问题引入**

​																		**2.idea可行性及现实意义分析**

​																		**3.所学知识拓展以备项目开发需要**

​														**二、所需数据特点分析、爬取数据工作**

​														**三、数据工程**

​														**四、建立模型、训练、调参及模型评估**

​														**五、预测结果**

​														**六、番外篇**—**项目衍生出的实际应用&项目之外的工作->打个广告**



















**一、问题引入、idea可行性及现实意义分析**

**问题引入**

​					Machine Learning is "answer questions using data".这句话作为这次课程设计的开头就十分的简洁精炼，概括了我们的目的，一言以蔽之，即用数据，采用机器学习的方法去分析，套用合适的模型来解决我们想要解决的问题。说实话在6.10之前我一直都还没有定下合适的题目，差不多是在6.11~6.12这两天才有了一定思路并去学习了所需的知识，才慢慢的把代码写的出来，虽然做的简单，但毕竟是自己原创的一个项目—**基于keras训练序列到序列（seq2seq）模型的自动对古诗、春联、横批（支持四字、五言、七言)的实例项目**。

​					为什么会有这个想法并愿意去做呢，是因为有现成的实例可以让我参照吗？说实在的，并没有。主要是前几天在github上看到一个用C++自动生成歌词器的项目，fork下来跑一下效果不错，突然想到自己平时也爱舞文弄墨的，喜欢<<滕王阁序>>的大气恢弘，也喜欢<<祭十二郎文>>的哀转久绝，不如也做一个AI对子王，做一回对王之王对穿肠，hh，说干就干，于是就有了这个项目。理想中应该要媲美<<唐伯虎点秋香>>中的对穿肠：

[![tXU19U.png](https://s1.ax1x.com/2020/06/12/tXU19U.png)](https://imgchr.com/i/tOeIBj)



**是否可行，有什么意义呢**

​			没错！阁下就是要做比肩对穿肠的文采斐然的AI自动对子机，想法看起来很强势，那需要什么知识呢？

​			前置知识：应用于自然语言处理的递归神经网络RNN的一种形式—LSTM模型

​			意义何在嘞？

​			1.帮助空有一番愿望却胸无点墨的笔者完成文艺青年的转型。

​			2.可以举一反三、利用LSTM写出脍炙人口的诗、歌曲，逢年过节也可以给亲戚写副文采斐然的对联，又锻炼了自己机器学习实战做项目的能力，岂不美哉？


​			2.交个期末课程设计报告？（不是，哪有这么应付)




**LSTM模型前置知识简介**

传统的神经网络并不能做到持久化及记忆化思考，看起来也像是一种巨大的弊端。例如，假设你希望对电影中的每个时间点的时间类型进行分类。传统的神经网络应该很难来处理这个问题——使用电影中先前的事件推断后续的事件。但RNN 解决了这个问题。RNN 是包含循环的网络，允许信息的持久化。
![tjGkFO.png](https://s1.ax1x.com/2020/06/13/tjGkFO.png)
但是，RNN也存在很明显的缺点。RNN 的关键点之一就是他们可以用来连接先前的信息到当前的任务上，例如使用过去的视频段来推测对当前段的理解。如果 RNN 可以做到这个，他们就变得非常有用。但是真的可以么？答案是，还有很多依赖因素。有时候，我们仅仅需要知道先前的信息来执行当前的任务。例如，我们有一个语言模型用来基于先前的词来预测下一个词。如果我们试着预测 “the clouds are in the sky” 最后的词，我们并不需要任何其他的上下文 —— 因此下一个词很显然就应该是 sky。在这样的场景中，相关的信息和预测的词位置之间的间隔是非常小的，RNN 可以学会使用先前的信息。但是同样会有一些更加复杂的场景。假设我们试着去预测“I grew up in France... I speak fluent French”最后的词。当前的信息建议下一个词可能是一种语言的名字，但是如果我们需要弄清楚是什么语言，我们是需要先前提到的离当前位置很远的 France 的上下文的。这说明相关信息和当前预测位置之间的间隔就肯定变得相当的大。不幸的是，在这个间隔不断增大时，RNN 会丧失学习到连接如此远的信息的能力。
**用人话来说就是：RNN 会受到短时记忆的影响。如果一条序列足够长，那它们将很难将信息从较早的时间步传送到后面的时间步。 因此，如果你正在尝试处理一段文本进行预测，RNN 可能从一开始就会遗漏重要信息。在反向传播期间，RNN 会面临梯度消失的问题。 梯度是用于更新神经网络的权重值，消失的梯度问题是当梯度随着时间的推移传播时梯度下降，如果梯度值变得非常小，就不会继续学习。**
而LSTM模型可以通过刻意的设计来避免长期依赖问题。记住长期的信息在实践中是  LSTM 的默认行为，而非需要付出很大代价才能获得的能力。


关于LSTM，算是一个比较复杂的模型，这里就简单的谈谈我的理解。

**1.直观来看，LSTM模型引入了sigmoid函数，这样的好处在于能够及时地更新或忘记信息，因为任何数乘以 0 都得 0，这部分信息就会剔除掉。同样的，任何数乘以 1 都得到它本身，这部分信息就会完美地保存下来。这样网络就能了解哪些数据是需要遗忘，哪些数据是需要保存。**

**2.个人认为，LSTM模型的创新性在于引入了细胞状态和门结构这两部分，使得较早时间步长的信息也能携带到较后时间步长的细胞中来，这克服了短时记忆的影响。信息的添加和移除我们通过“门”结构来实现，“门”结构在训练过程中会去学习该保存或遗忘哪些信息。**

具体展开来说，细胞状态在一个神经网络层中只进行运算两次，**第一次是与遗忘向量相乘，第二次是将该值与输入门的输出值逐点相加，将神经网络发现的新信息更新到细胞状态中去。此后，作为下一个隐藏层的输入，保持不变。**

下面给一张动图来说明这一点，这张动图做的很好，有一说一。

![tja4AI.gif](https://s1.ax1x.com/2020/06/13/tja4AI.gif)

而对于门结构，则分为遗忘门、输入门、以及输出门

遗忘门主要是决定应丢弃或保留哪些信息。来自前一个隐藏状态的信息和当前输入的信息同时传递到 sigmoid 函数中去，输出值介于 0 和 1 之间，越接近 0 意味着越应该丢弃，越接近 1 意味着越应该保留。

哦对了，应该附带一个sigmoid函数的图像复习

[![tjdNPP.png](https://s1.ax1x.com/2020/06/13/tjdNPP.png)](https://imgchr.com/i/tjdNPP)

前面也说到sigmoid函数十分好用，因为有了这个函数，原本的神经网络就可以模拟并无限接近任何非线性的模型

下图是遗忘门结构图示：
[![tjdsVs.gif](https://s1.ax1x.com/2020/06/13/tjdsVs.gif)](https://imgchr.com/i/tjdsVs)

而输入门则用于更新细胞状态。首先将前一层隐藏状态的信息和当前输入的信息传递到 sigmoid 函数中去。将值调整到 0~1 之间来决定要更新哪些信息。0 表示不重要，1 表示重要。

其次还要将前一层隐藏状态的信息和当前输入的信息传递到 tanh 函数中去，创造一个新的侯选值向量。最后将 sigmoid 的输出值与 tanh 的输出值相乘，sigmoid 的输出值将决定 tanh 的输出值中哪些信息是重要且需要保留下来的。

[![tjw0W6.gif](https://s1.ax1x.com/2020/06/13/tjw0W6.gif)](https://imgchr.com/i/tjw0W6)

最后是输出门，输出门主要用来确定下一个隐藏状态的值，隐藏状态包含了先前输入的信息。首先，我们将前一个隐藏状态和当前输入传递到 sigmoid 函数中，然后将新得到的细胞状态传递给 tanh 函数。

最后将 tanh 的输出与 sigmoid 的输出相乘，以确定隐藏状态应携带的信息。再将隐藏状态作为当前细胞的输出，把新的细胞状态和新的隐藏状态传递到下一个时间步长中去。

[![tjwTOg.gif](https://s1.ax1x.com/2020/06/13/tjwTOg.gif)](https://imgchr.com/i/tjwTOg)



**设计思路**

讲完了基本依赖的模型原理，那就讲一讲实际套用时的设计思路吧：

由于下面还会展开详细的思路，这里主要简述一下大致的轮廓，使用TensorFlow进行建模，TensorFlow就无需多言，是这个领域目前最活跃的框架。接着根据样本数据产生LSTM输入数据和结果值；定义LSTM的模型以及损失函数；将训练数据喂给TensorFlow用来训练模型。首先做数据工程，数据字段调整、清洗，统计所有出现的汉字进行编码，进行数据转换；定义LSTM模型、tf.gradients 计算梯度调参；结束训练，评估模型


 **二、所需数据特点分析、爬取数据工作**
 
 
 俗话说，巧妇难为无米之炊，我们需要有足够多的数据，当然这些数据还需要保证质量，这里我选取的是《诗经》、《古诗十九首》、《唐诗三百首》、王国维的《人间词话》，常用对联对偶句共**1630194**字，数据获取方式采取Python 网络爬虫，关键代码如下：

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#Author:glm233 22920182204174

#导入基本必备库
import scrapy
import logging
import re

class CoupletSpider(scrapy.Spider):
    name = 'CoupletSpider'
    start_urls = ['http://blog.sina.com.cn/s/articlelist_1195052695_2_%d.html' % (i) for i in range(1, 20)]


    def parse(self, response):
        for href in response.css('.atc_title a ::attr(href)'):
            yield response.follow(href.extract(), self.parse_couplets)

    #词句分析，爬取数据写入文件
    def parse_verse(self, response):
        title = response.css('.articalTitle h2 ::text').extract_first()
        lines = response.css('.articalContent::text, .articalContent *::text').extract()
        output_file = open('output/' + title + '.txt', 'wb')
        mid = u'〓'
        end = u'◎'
        for line in lines:
            if (not mid in line) or (not end in line):
                continue
            line = line.replace(end, '').replace('|', '')
            words = re.split(r"[\u200b\s]+", line, flags=re.UNICODE)
            for word in words:
                if len(word.strip()) == 0:
                    continue
                couplet = word.split(mid)
                if len(couplet) != 2:
                    logging.warning("Error while process " + word)
                    continue
                up, down = couplet
                if (not up) or (not down) or len(up) != len(down):
                    logging.warning("Error while process " + word)
                    continue
                output_file.write((up + u'\n').encode('utf8'))
                output_file.write((down + u'\n').encode('utf8'))
        output_file.close()

部分爬取数据概览

**古诗十九首部分**

行行重行行

行行重行行，与君生别离。

相去万余里，各在天一涯。

道路阻且长，会面安可知？

胡马依北风，越鸟巢南枝。

相去日已远，衣带日已缓。

浮云蔽白日，游子不顾返。

思君令人老，岁月忽已晚。

弃捐勿复道，努力加餐饭

**迢迢牵牛星**  

迢迢牵牛星，皎皎河汉女。

纤纤擢素手，札札弄机杼。

终日不成章，泣涕零如雨。

河汉清且浅，相去复几许！

盈盈一水间，脉脉不得语。

......

**诗经部分**

樛木 

南有樛木，葛藟累之。乐只君子，福履绥之。

南有樛木，葛藟荒之。乐只君子，福履将之。

南有樛木，葛藟萦之。乐只君子，福履成之。

芣苡 

采采芣苡，薄言采之。采采芣苡，薄言有之。

采采芣苡，薄言掇之。采采芣苡，薄言捋之。

采采芣苡，薄言袺之。采采芣苡，薄言襭之。

......

**对联部分**

白头翁骑海马赴常山挥大戟怒战草蔻百合	何首乌驾河豚入大海操仙茅逼杀木贼千年

人说之人被人说之人说人人被说不如不说	官管之管被官管之管管管管被管不如不管

红面关黑面张白面子龙面面护着刘先生	奸心曹雄心瑜阴心董卓心心夺取汉江山

游西湖提锡壶锡壶掉西湖惜乎锡湖	过南平卖蓝瓶蓝瓶得南平难得蓝瓶

金水河边金线柳金线柳穿金鱼口	玉栏杆外玉簪花玉簪花插玉人头

开口便笑笑古笑今凡事付之一笑	大肚能容容天容地与己何所不容

童子看橡一二三四五六七八九十	先生讲命甲乙丙丁戊己庚辛壬癸

暗室贞邪谁见，忽而万口喧传	自心善恶炯然，凛于四王考校

葆真莫如少思，寡过莫如省事	善应莫如收心，解谬莫如澹志

才有力以胜蝶，本无心而引莺	半叶舒而岩暗，一花散而峰明

才智英敏者，宜以学问摄其躁	气节激昂者，当以德性融其偏

草色伴河桥，锦缆晓牵三竺雨	花阴连野寺，布帆晴挂六桥烟


草色遍溪桥，醉得蜻蜓春翅软	花风通驿路，迷来蝴蝶晓魂香

茶取色臭俱佳，行家偏嫌味苦	香须冲淡为雅，幽人最忌烟浓

醇醪百斛，不如一味太和之汤	良药千包，不如一服清凉之散

词人半肩行李，收拾秋水春云	深宫一世梳妆，恼乱晚花新柳

带雨有时种竹，关门无事锄花	拈笔闲删旧句，汲泉几试新茶

待小人不难于严，而难于不恶	待君子不难于恭，而难于有礼

澹泊之守，须从秾艳场中试来	镇定之操，还向纷纭境上勘过

当场笑语，尽如形骸外之好人	背地风波，谁是意气中之烈士

肝胆相照，欲与天下共分秋月	意气相许，欲与天下共坐春风

高僧筒里送信，突地天花坠落	韵妓扇头寄画，隔江山雨飞来

攻人之恶毋太严，要思其堪受	教人以善莫过高，当原其可从

贺函伯坐径山竹里，须眉皆碧	王长公龛杜鹃楼下，云母都红

**以上就是爬取数据的部分，至此，数据收集工作基本完成**

**三、数据工程**

接下来是数据的一些预处理工作，emm数据工程是我自己起的一个名字，具体可以参看我的这一篇《论机器学习中数据的重要性》中我自己对数据工程的定义
https://newcoder-glm.blog.csdn.net/article/details/106733445

In [120]:
#常规操作导入基本库
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

from smart_open import smart_open
from tensorflow.python.framework import ops
ops.reset_default_graph()



首先，我们将加载文本文件并将其转换为整数以供网络使用。这里我创建了两个字典来将字符与整数进行转换。将字符编码为整数使其更容易在网络中用作输入。

In [103]:
with open(r'C:\Users\ASUS\Desktop\模式识别期末课设报告\模式识别期末课设报告数据集.txt', 'r',encoding='utf-8') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

让我们查看文件写入的前一百个字符，确保一切准确无误。

In [104]:
text[:100]

'阿房久已灭\t阁道遂成墟\n白头翁骑海马赴常山挥大戟怒战草蔻百合\t何首乌驾河豚入大海操仙茅逼杀木贼千年\n人说之人被人说之人说人人被说不如不说\t官管之管被官管之管管管管被管不如不管\n红面关黑面张白面子龙面面'

在这里我们可以看到字符被编码成整数。

In [105]:
encoded[:100]

array([6203, 1883,   69, 1460, 3172,    0, 6164, 5828, 5820, 1866, 1026,
          1, 3635, 1068, 4438, 6535, 2927, 6504, 5560, 1493, 1346, 1998,
       1060, 1874, 1686, 1871, 4776, 4942, 3636,  666,    0,  198, 6475,
         73, 6520, 2824, 5460,  370, 1060, 2927, 2129,  144, 4748, 5817,
       2360, 2347, 5515,  568, 1512,    1,  124, 5395,   72,  124, 5198,
        124, 5395,   72,  124, 5395,  124,  124, 5198, 5395,   35, 1099,
         35, 5395,    0, 1250, 4099,   72, 4099, 5198, 1250, 4099,   72,
       4099, 4099, 4099, 4099, 5198, 4099,   35, 1099,   35, 4099,    1,
       4251, 6321,  378, 6802, 6321, 1581, 3635, 6321, 1216, 6857, 6321,
       6321])

由于网络处理的是单个字符，这就类似于分类问题，即我们试图从前面的文本中预测下一个字符。以下是我们的网络需要选择的“课程”。

In [122]:
#打印句子数
len(vocab)

125032


**作mini-batches工作**

Mini-batching 是一个一次训练数据集的一小部分，而不是整个训练集的技术。它可以使内存较小、不能同时训练整个数据集的电脑也可以训练模型。

我们希望得到的序列是一些所需的序列步骤的多个序列。



In [107]:
def get_batches(arr, batch_size, n_steps):
    '''获得训练轮数及每轮步数
       
       参数解释
       ---------
       arr: 你想要进行批处理的数组
       batch_size: 一轮训练的序列数
       n_steps: 每次训练的顺序步骤数
    '''
    # 获得每个批次的字符数和我们可以生产的批次数
    characters_per_batch = batch_size * n_steps
    n_batches = len(arr) // characters_per_batch
    
    # 只保留足够的字符来完成一轮训练
    arr = arr[: n_batches * characters_per_batch]
    
    # 重新设置为batch_size行
    arr = arr.reshape(batch_size, -1)
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # y目标，移动了一步
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        
        yield x, y

在这里，使用10和50个顺序步骤的每轮大小。

In [108]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [109]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[6203 1883   69 1460 3172    0 6164 5828 5820 1866]
 [4465  184    0 1598 2380  166 2436 3635 6574 6372]
 [   1 1346   55 2223 1240  893 2219 5869    0 4776]
 [1376  567 4648  101 2380 1620    1  649 1304 4426]
 [   1 3496 6319 3693 3669 1259    0 5981 2332 5390]
 [6708 4562 1973 2347    0 4425 4419 3089 5286 4990]
 [  99 5766    0  895 1068 3635 2221 2185    1   66]
 [   0 1061   33 1078 4977 2686    1 5713 4520  198]
 [ 715 2332 2610 1068    1  916 2738 3426  469 5566]
 [1750 1750    1  111 2219 5694 6504 1259    0 5807]]

y
 [[1883   69 1460 3172    0 6164 5828 5820 1866 1026]
 [ 184    0 1598 2380  166 2436 3635 6574 6372    1]
 [1346   55 2223 1240  893 2219 5869    0 4776  417]
 [ 567 4648  101 2380 1620    1  649 1304 4426 2950]
 [3496 6319 3693 3669 1259    0 5981 2332 5390 1985]
 [4562 1973 2347    0 4425 4419 3089 5286 4990    1]
 [5766    0  895 1068 3635 2221 2185    1   66 5847]
 [1061   33 1078 4977 2686    1 5713 4520  198   63]
 [2332 2610 1068    1  916 2738 3426  

如果正确地实现了get_batch，那么上面的输出应该与下面类似
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
尽管可能运行结果的得到确切的数字会有所不同。检查以确保y数据被移动了一步。

​ **四、建立模型、训练、调参及模型评估**

**建立模型**

下面是我们将建立网络的地方。我们将把它分成几个部分，这样就更容易推理出每一点。然后我们可以把它们连接到整个网络。



**输入**

首先，我们将创建输入占位符。像往常一样，我们需要为训练数据和目标预留位置。我们还将为dropout层创建一个名为“keep_prob”的占位符。这是一个标量，这是一个0-D张量。要创建标量，需要创建一个占位符，而不给它一个大小。


In [110]:
def build_inputs(batch_size, num_steps):
    ''' 为输入、目标和退出定义占位符
    
        
        
    '''
    #声明占位符
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # 保留概率占位符为Dropout层
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

**LSTM 单元部分介绍**

在这里，我们将创建隐藏层中使用的LSTM单元格。我们将使用这个单元格作为RNN的构建块。这里我们并没有定义RNN，只是我们在隐藏层中使用的单元格类型。在这里，我们将创建隐藏层中使用的LSTM单元格。我们将使用这个单元格作为RNN的构建块。这里我们并没有定义RNN，只是我们在隐藏层中使用的单元格类型。

首先创建一个基本的LSTM单元格

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

其中' num_units '是单元格中隐藏层的单元数。然后我们可以通过包装它来添加dropou

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
传入一个单元格，它将自动向输入或输出添加dropout。最后，我们可以使用[' tf.contrib.rnn.MultiRNNCell '](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell)将LSTM单元堆叠成层。这样，您传入一个单元格列表，它将把一个单元格的输出发送到下一个单元格。在之前的TensorFlow 1.0中，你可以这样做

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

如果你熟悉Python，这看起来可能有点奇怪，因为这将创建一个相同的cell对象列表。然而，TensorFlow 1.0将为所有“cell”对象创建不同的权重矩阵。但是，从TensorFlow 1.1开始，实际上需要在列表中创建新的cell对象。要让它在TensorFlow 1.1中工作，它应该是这样的

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

尽管这实际上是堆叠在一起的多个LSTM单元，但您可以将多个层视为一个单元。


我们还需要创建所有为零的初始单元格状态。可以这样做

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

下面，我们实现' build_lstm '函数来创建这些LSTM单元格和初始状态。

In [111]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
       参数解释
       ---------
       arr: 你想要进行批处理的数组
       batch_size: 一轮训练的序列数
       n_steps: 每次训练的顺序步骤数

    '''
    ### 构建LSTM单元
    
    def build_cell(lstm_size, keep_prob):
        # 使用基本的LSTM单元格
        lstm = tf.contrib.rnn.LayerNormBasicLSTMCell(lstm_size,activation = tf.nn.softsign)
        # 将dropout添加到单元格输出
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop

    
    # 堆叠多个LSTM层，用于深度学习
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

**RNN输出**

这里我们将创建输出层。我们需要将RNN单元的输出连接到一个softmax输出的全连接层。softmax输出给我们一个概率分布，我们可以使用它来预测下一个字符，因此我们希望这一层的大小为C，即文本中类/字符的数量。


In [112]:
def build_output(lstm_output, in_size, out_size):
    ''' 建立一个softmax层，返回softmax输出和日志
    
        参数解释
        ---------
        
        lstm_output: 来自LSTM层的输出张量列表
        in_size: 输入张量的大小，例如，LSTM单元的大小
        out_size: softmax层的大小
    
    '''

    # 重新调整输出，使其成为一组行，每个步骤对应每个序列对应一行。
    # 在轴1(列)上连接lstm_output
    seq_output = tf.concat(lstm_output, axis=1)
    # 将seq_output重新定义为一个包含lstm_size列的2D张量
    x = tf.reshape(seq_output, [-1, in_size])
    
    # 连接RNN输出到softmax层
    with tf.variable_scope('softmax'):
        # 在这里创建权重和偏差变量
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # 由于输出是RNN单元格输出的一组行，因此logits将是一组logit输出，每个步骤和序列对应一行
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # 使用softmax获得预测字符的概率
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits,softmax_w,softmax_b

**训练损失**

接下来是训练损失。在此基础上，计算了最小交叉熵损失。

In [113]:
def build_loss(logits, targets, lstm_size, num_classes,softmax_w,softmax_b):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: 最终完全连接层Logits
        targets: 监督学习的目标
        lstm_size: LSTM隐藏单元的数量
        num_classes: 目标中的类数目
        
    '''
    
    # 单热编码目标和重塑以匹配日志，每个序列每个步骤一行
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax交叉熵损失
    
    #vars   = tf.trainable_variables() 
    #lossL2 = tf.add_n([ tf.nn.l2_loss(v) for v in vars ]) * 0.001
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y_reshaped))
    #loss= + 0.01*tf.nn.l2_loss(softmax_w) + 0.01*tf.nn.l2_loss(softmax_b)
    
    
    
    return loss

**设置优化器**

这里我们构建优化器。正常的网络神经网络有渐变爆炸和消失的问题。LSTMs修复了消失问题，但是梯度仍然可以无限制地增长。为了解决这个问题，我们可以将梯度剪辑到某个阈值以上。也就是说，如果一个梯度大于阈值，我们将它设置为阈值。这将确保梯度不会变得过大。然后我们使用一个adam优化器的学习步骤。

In [114]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' 建立优化器，使用梯度下降。
    
        参数解释:
        loss: 神经网络损失
        learning_rate: 优化器的学习率
    
    '''
    
    # 优化训练，使用梯度下降控制增长梯度
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

**建立网络**

现在我们可以把所有的部分放在一起，为网络构建一个类。为了在LSTM单元中实际运行数据，我们将使用tf.nn.dynamic_rnn 函数。(https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn)。这个函数将为我们在LSTM单元间传递隐藏状态和单元状态。对于Mini_batch中的每个序列，它在每个步骤中返回每个LSTM单元的输出。它还提供了最终的LSTM状态。我们希望将此状态保存为' final_state '，以便在下一个小批处理运行时将其传递给第一个LSTM单元。“tf.nn。dynamic_rnn '，我们传递单元格和初始状态，我们从' build_lstm '，以及我们的输入序列。同样，我们需要在进入RNN之前对输入进行一次编码。


In [115]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=4, sampling=False):
    
        # 当我们稍后使用这个网络进行采样时，我们会传入一次一个字符，所以提供了一个选项
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        with tf.name_scope('Inputs'):
        # 构建输入占位符张量
            self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # 构建LSTM单元
        with tf.name_scope('Lstm'):
            cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### 通过RNN层运行数据
        # 首先，对输入标记进行one-hot编码
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # 使用tf.n .dynamic_rnn运行RNN中的每个序列步骤
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # 获得softmax的预测和日志
        self.prediction, self.logits,self.softmax_w,self.softmax_b = build_output(outputs, lstm_size, num_classes)
        
        # 损失和优化(与梯度下降)
        with tf.name_scope('Loss'):
            self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes,self.softmax_w,self.softmax_b)
        with tf.name_scope('SGD'):
            self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

**超参数设置**

这里是网络的超参数：

* `batch_size` - 一次通过网络的序列数。
* `num_steps` -网络训练所依据的序列中的字符数。通常情况下，网络会学习到更多的长期依赖关系。但是它需要更长的训练时间。100在这里是一个很好的数字。
* `lstm_size` - 隐藏层中的单元数。
* `num_layers` - 要使用的LSTM隐藏层数
* `learning_rate` - 训练时设置的学习率
* `keep_prob` - dropout层节点保留率


**调优中的一些心得**

**监控验证损失和训练损失**

需要跟踪的最重要数量是训练损失(在训练期间打印)和验证损失(在RNN对验证数据运行时偶尔打印一次(默认每1000次迭代))之间的差异。防止**过拟合**(如果你的训练损失远低于验证损失)解决这个问题的方法是减小网络大小，或者增加dropout。例如，你可以尝试从0。5中退出，等等。h或者是**不拟合**:如果训练/验证损失是相等的，这时增加模型的大小(层的数量或者每层的原始神经元数量)

**参数选取及模型构建**

控制模型的两个最重要的参数是lstm_size和num_layers。我调试的时候发现应该使用2/3的num_layers。至于lstm_size，则可以根据自己拥有的数据量进行调整。

在深度学习中，使用许多不同的超参数设置运行许多不同的模型是很常见的，并最终采用能够提供最佳验证性能的检查点。

顺便说一下，训练和验证分割的大小比例也是参数。应当确保验证集中有相当数量的数据，否则验证性能会很麻烦，信息也不是很丰富。

In [116]:
batch_size = 256    # 每轮序列数
num_steps = 120    # 每轮训练的顺序步骤数
lstm_size = 512     # LSTM隐藏层层数
num_layers = 2         # LSTM层数
learning_rate = 0.003    # 学习率
keep_prob = 0.5     # Dropout层节点保留率

**模型训练**

这是典型的训练代码，将输入和目标传递到网络中，然后运行优化器。在这里，我们还得到了mini_batch处理的最终LSTM状态。然后，我们将该状态传递回网络，以便下一个训练轮处理可以延续上一个训练轮处理的状态。每隔一段时间(由' save_every_n '设置)我保存一个检查点。


In [ ]:
epochs = 30
# 每N次迭代打印损失
print_every_n = 1

# 每N次迭代保存一次
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 使用下面一行代码加载检查点并恢复训练
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # 训练网络
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            if (counter % print_every_n == 0):
                end = time.time()
                print('Epoch: {}/{}... '.format(e+1, epochs),
                      'Training Step: {}... '.format(counter),
                      'Training loss: {:.4f}... '.format(batch_loss),
                      '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/Step{}_LstmSize{}_Loss{}.ckpt".format(counter, lstm_size,batch_loss))
    
    #saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    saver.save(sess, "checkpoints/Step{}_LstmSize{}_Loss{}.ckpt".format(counter, lstm_size,batch_loss))

**训练结果：**
在单个P100 GPU上运行，此示例大约需要10分钟才能跑完。
由于我的电脑过于渣，所以放在Openbayes上运行，只能贴一个在平台上运行的结果了:

Epoch: 1/30...  Training Step: 1...  Training loss: 9.0058...  1.7413 sec/batch
Epoch: 1/30...  Training Step: 2...  Training loss: 8.7586...  1.4611 sec/batch
Epoch: 1/30...  Training Step: 3...  Training loss: 8.1588...  1.4039 sec/batch
Epoch: 1/30...  Training Step: 4...  Training loss: 7.8304...  1.4338 sec/batch
Epoch: 1/30...  Training Step: 5...  Training loss: 7.5796...  1.5521 sec/batch
Epoch: 1/30...  Training Step: 6...  Training loss: 7.3030...  1.3840 sec/batch
Epoch: 1/30...  Training Step: 7...  Training loss: 7.1113...  1.4682 sec/batch
Epoch: 1/30...  Training Step: 8...  Training loss: 6.9905...  1.4619 sec/batch
Epoch: 1/30...  Training Step: 9...  Training loss: 6.9037...  1.5316 sec/batch
Epoch: 1/30...  Training Step: 10...  Training loss: 6.8688...  1.3987 sec/batch
Epoch: 1/30...  Training Step: 11...  Training loss: 6.8411...  1.4282 sec/batch
Epoch: 1/30...  Training Step: 12...  Training loss: 6.8224...  1.3405 sec/batch
Epoch: 1/30...  Training Step: 13...  Training loss: 6.8446...  1.5365 sec/batch
Epoch: 1/30...  Training Step: 14...  Training loss: 6.8260...  1.4937 sec/batch
Epoch: 1/30...  Training Step: 15...  Training loss: 6.8011...  1.5297 sec/batch
Epoch: 1/30...  Training Step: 16...  Training loss: 6.7902...  1.4366 sec/batch
Epoch: 1/30...  Training Step: 17...  Training loss: 6.7883...  1.4100 sec/batch
Epoch: 1/30...  Training Step: 18...  Training loss: 6.7714...  1.4713 sec/batch
Epoch: 1/30...  Training Step: 19...  Training loss: 6.7336...  1.5085 sec/batch
Epoch: 1/30...  Training Step: 20...  Training loss: 6.7299...  1.4728 sec/batch
Epoch: 1/30...  Training Step: 21...  Training loss: 6.7293...  1.4387 sec/batch
Epoch: 1/30...  Training Step: 22...  Training loss: 6.7390...  1.3046 sec/batch
Epoch: 1/30...  Training Step: 23...  Training loss: 6.7456...  1.3737 sec/batch
Epoch: 1/30...  Training Step: 24...  Training loss: 6.7739...  1.4540 sec/batch
Epoch: 1/30...  Training Step: 25...  Training loss: 6.7623...  1.3940 sec/batch
Epoch: 1/30...  Training Step: 26...  Training loss: 6.7664...  1.4498 sec/batch
Epoch: 1/30...  Training Step: 27...  Training loss: 6.7558...  1.4505 sec/batch
Epoch: 1/30...  Training Step: 28...  Training loss: 6.7371...  1.4009 sec/batch
Epoch: 1/30...  Training Step: 29...  Training loss: 6.7409...  1.4431 sec/batch
Epoch: 1/30...  Training Step: 30...  Training loss: 6.7188...  1.5942 sec/batch
Epoch: 1/30...  Training Step: 31...  Training loss: 6.7060...  1.6128 sec/batch
Epoch: 1/30...  Training Step: 32...  Training loss: 6.6926...  1.5623 sec/batch
Epoch: 1/30...  Training Step: 33...  Training loss: 6.6953...  1.3549 sec/batch
Epoch: 1/30...  Training Step: 34...  Training loss: 6.7017...  1.3186 sec/batch
Epoch: 1/30...  Training Step: 35...  Training loss: 6.6940...  1.6040 sec/batch
Epoch: 1/30...  Training Step: 36...  Training loss: 6.6951...  1.4684 sec/batch
Epoch: 1/30...  Training Step: 37...  Training loss: 6.7278...  1.5126 sec/batch
Epoch: 1/30...  Training Step: 38...  Training loss: 6.7091...  1.5029 sec/batch
Epoch: 1/30...  Training Step: 39...  Training loss: 6.7260...  1.4993 sec/batch
Epoch: 1/30...  Training Step: 40...  Training loss: 6.7204...  1.4479 sec/batch
Epoch: 1/30...  Training Step: 41...  Training loss: 6.7033...  1.4868 sec/batch
Epoch: 1/30...  Training Step: 42...  Training loss: 6.7156...  1.3338 sec/batch
Epoch: 1/30...  Training Step: 43...  Training loss: 6.6944...  1.4602 sec/batch
Epoch: 1/30...  Training Step: 44...  Training loss: 6.6862...  1.3507 sec/batch
Epoch: 1/30...  Training Step: 45...  Training loss: 6.6923...  1.5047 sec/batch
Epoch: 1/30...  Training Step: 46...  Training loss: 6.7013...  1.3389 sec/batch
Epoch: 1/30...  Training Step: 47...  Training loss: 6.6870...  1.3341 sec/batch
Epoch: 1/30...  Training Step: 48...  Training loss: 6.6944...  1.3070 sec/batch
Epoch: 1/30...  Training Step: 49...  Training loss: 6.6627...  1.3130 sec/batch
Epoch: 1/30...  Training Step: 50...  Training loss: 6.6911...  1.4491 sec/batch
Epoch: 1/30...  Training Step: 51...  Training loss: 6.6802...  1.4654 sec/batch
Epoch: 1/30...  Training Step: 52...  Training loss: 6.7001...  1.3772 sec/batch
Epoch: 1/30...  Training Step: 53...  Training loss: 6.7078...  1.3876 sec/batch
Epoch: 1/30...  Training Step: 54...  Training loss: 6.6742...  1.3175 sec/batch
Epoch: 1/30...  Training Step: 55...  Training loss: 6.7043...  1.3174 sec/batch
Epoch: 1/30...  Training Step: 56...  Training loss: 6.6959...  1.3214 sec/batch
Epoch: 1/30...  Training Step: 57...  Training loss: 6.6865...  1.3707 sec/batch
Epoch: 1/30...  Training Step: 58...  Training loss: 6.6952...  1.3350 sec/batch
Epoch: 1/30...  Training Step: 59...  Training loss: 6.6729...  1.4319 sec/batch
Epoch: 1/30...  Training Step: 60...  Training loss: 6.6861...  1.5329 sec/batch
Epoch: 1/30...  Training Step: 61...  Training loss: 6.6699...  1.4438 sec/batch
Epoch: 1/30...  Training Step: 62...  Training loss: 6.6710...  1.5462 sec/batch
Epoch: 2/30...  Training Step: 63...  Training loss: 6.8626...  1.3105 sec/batch
Epoch: 2/30...  Training Step: 64...  Training loss: 6.6649...  1.3759 sec/batch
Epoch: 2/30...  Training Step: 65...  Training loss: 6.6683...  1.4048 sec/batch
Epoch: 2/30...  Training Step: 66...  Training loss: 6.7001...  1.4245 sec/batch
Epoch: 2/30...  Training Step: 67...  Training loss: 6.6654...  1.4350 sec/batch
Epoch: 2/30...  Training Step: 68...  Training loss: 6.6635...  1.4189 sec/batch
Epoch: 2/30...  Training Step: 69...  Training loss: 6.6587...  1.4847 sec/batch
Epoch: 2/30...  Training Step: 70...  Training loss: 6.6743...  1.4342 sec/batch
Epoch: 2/30...  Training Step: 71...  Training loss: 6.6608...  1.4083 sec/batch
Epoch: 2/30...  Training Step: 72...  Training loss: 6.6629...  1.4429 sec/batch
Epoch: 2/30...  Training Step: 73...  Training loss: 6.6588...  1.2454 sec/batch
Epoch: 2/30...  Training Step: 74...  Training loss: 6.6534...  1.4102 sec/batch
Epoch: 2/30...  Training Step: 75...  Training loss: 6.6831...  1.3825 sec/batch
Epoch: 2/30...  Training Step: 76...  Training loss: 6.6651...  1.4777 sec/batch
Epoch: 2/30...  Training Step: 77...  Training loss: 6.6610...  1.3177 sec/batch
Epoch: 2/30...  Training Step: 78...  Training loss: 6.6619...  1.3632 sec/batch
Epoch: 2/30...  Training Step: 79...  Training loss: 6.6761...  1.3539 sec/batch
Epoch: 2/30...  Training Step: 80...  Training loss: 6.6722...  1.6130 sec/batch
Epoch: 2/30...  Training Step: 81...  Training loss: 6.6380...  1.3406 sec/batch
Epoch: 2/30...  Training Step: 82...  Training loss: 6.6400...  1.4736 sec/batch
Epoch: 2/30...  Training Step: 83...  Training loss: 6.6509...  1.2893 sec/batch
Epoch: 2/30...  Training Step: 84...  Training loss: 6.6646...  1.4444 sec/batch
Epoch: 2/30...  Training Step: 85...  Training loss: 6.6771...  1.4946 sec/batch
Epoch: 2/30...  Training Step: 86...  Training loss: 6.7086...  1.3730 sec/batch
Epoch: 2/30...  Training Step: 87...  Training loss: 6.6989...  1.3083 sec/batch
Epoch: 2/30...  Training Step: 88...  Training loss: 6.7043...  1.3372 sec/batch
Epoch: 2/30...  Training Step: 89...  Training loss: 6.6974...  1.3411 sec/batch
Epoch: 2/30...  Training Step: 90...  Training loss: 6.6864...  1.4619 sec/batch
Epoch: 2/30...  Training Step: 91...  Training loss: 6.6901...  1.5257 sec/batch
Epoch: 2/30...  Training Step: 92...  Training loss: 6.6723...  1.4316 sec/batch
Epoch: 2/30...  Training Step: 93...  Training loss: 6.6676...  1.4249 sec/batch
Epoch: 2/30...  Training Step: 94...  Training loss: 6.6546...  1.4361 sec/batch
Epoch: 2/30...  Training Step: 95...  Training loss: 6.6562...  1.3635 sec/batch
Epoch: 2/30...  Training Step: 96...  Training loss: 6.6634...  1.6874 sec/batch
Epoch: 2/30...  Training Step: 97...  Training loss: 6.6543...  1.3561 sec/batch
Epoch: 2/30...  Training Step: 98...  Training loss: 6.6595...  1.3142 sec/batch
Epoch: 2/30...  Training Step: 99...  Training loss: 6.6881...  1.3381 sec/batch
Epoch: 2/30...  Training Step: 100...  Training loss: 6.6750...  1.4198 sec/batch
Epoch: 2/30...  Training Step: 101...  Training loss: 6.6933...  1.4793 sec/batch
Epoch: 2/30...  Training Step: 102...  Training loss: 6.6831...  1.5916 sec/batch
Epoch: 2/30...  Training Step: 103...  Training loss: 6.6684...  1.3703 sec/batch
Epoch: 2/30...  Training Step: 104...  Training loss: 6.6891...  1.5139 sec/batch
Epoch: 2/30...  Training Step: 105...  Training loss: 6.6658...  1.4355 sec/batch
Epoch: 2/30...  Training Step: 106...  Training loss: 6.6540...  1.5690 sec/batch
Epoch: 2/30...  Training Step: 107...  Training loss: 6.6656...  1.4744 sec/batch
Epoch: 2/30...  Training Step: 108...  Training loss: 6.6694...  1.4722 sec/batch
Epoch: 2/30...  Training Step: 109...  Training loss: 6.6629...  1.3279 sec/batch
Epoch: 2/30...  Training Step: 110...  Training loss: 6.6676...  1.3884 sec/batch
Epoch: 2/30...  Training Step: 111...  Training loss: 6.6365...  1.3415 sec/batch
Epoch: 2/30...  Training Step: 112...  Training loss: 6.6609...  1.3883 sec/batch
Epoch: 2/30...  Training Step: 113...  Training loss: 6.6518...  1.2704 sec/batch
Epoch: 2/30...  Training Step: 114...  Training loss: 6.6721...  1.4289 sec/batch
Epoch: 2/30...  Training Step: 115...  Training loss: 6.6814...  1.4109 sec/batch
Epoch: 2/30...  Training Step: 116...  Training loss: 6.6402...  1.4485 sec/batch
Epoch: 2/30...  Training Step: 117...  Training loss: 6.6721...  1.3290 sec/batch
Epoch: 2/30...  Training Step: 118...  Training loss: 6.6635...  1.3218 sec/batch
Epoch: 2/30...  Training Step: 119...  Training loss: 6.6914...  1.4364 sec/batch
Epoch: 2/30...  Training Step: 120...  Training loss: 6.6681...  1.4570 sec/batch
Epoch: 2/30...  Training Step: 121...  Training loss: 6.6466...  1.5049 sec/batch
Epoch: 2/30...  Training Step: 122...  Training loss: 6.6637...  1.4804 sec/batch
Epoch: 2/30...  Training Step: 123...  Training loss: 6.6452...  1.3979 sec/batch
Epoch: 2/30...  Training Step: 124...  Training loss: 6.6411...  1.4732 sec/batch
Epoch: 3/30...  Training Step: 125...  Training loss: 6.6503...  1.4575 sec/batch
Epoch: 3/30...  Training Step: 126...  Training loss: 6.6309...  1.4475 sec/batch
Epoch: 3/30...  Training Step: 127...  Training loss: 6.6260...  1.3726 sec/batch
Epoch: 3/30...  Training Step: 128...  Training loss: 6.6654...  1.4018 sec/batch
Epoch: 3/30...  Training Step: 129...  Training loss: 6.6279...  1.4389 sec/batch
Epoch: 3/30...  Training Step: 130...  Training loss: 6.7375...  1.5067 sec/batch
Epoch: 3/30...  Training Step: 131...  Training loss: 6.7092...  1.4024 sec/batch
Epoch: 3/30...  Training Step: 132...  Training loss: 6.7067...  1.2973 sec/batch
Epoch: 3/30...  Training Step: 133...  Training loss: 6.6494...  1.5016 sec/batch
Epoch: 3/30...  Training Step: 134...  Training loss: 6.6552...  1.3189 sec/batch
Epoch: 3/30...  Training Step: 135...  Training loss: 6.6695...  1.3322 sec/batch
Epoch: 3/30...  Training Step: 136...  Training loss: 6.6738...  1.4216 sec/batch
Epoch: 3/30...  Training Step: 137...  Training loss: 6.6924...  1.3080 sec/batch
Epoch: 3/30...  Training Step: 138...  Training loss: 6.6608...  1.4263 sec/batch
Epoch: 3/30...  Training Step: 139...  Training loss: 6.6503...  1.5389 sec/batch
Epoch: 3/30...  Training Step: 140...  Training loss: 6.6534...  1.4683 sec/batch
Epoch: 3/30...  Training Step: 141...  Training loss: 6.6728...  1.3653 sec/batch
Epoch: 3/30...  Training Step: 142...  Training loss: 6.6753...  1.4776 sec/batch
Epoch: 3/30...  Training Step: 143...  Training loss: 6.6346...  1.3241 sec/batch
Epoch: 3/30...  Training Step: 144...  Training loss: 6.6335...  1.4164 sec/batch
Epoch: 3/30...  Training Step: 145...  Training loss: 6.6392...  1.4538 sec/batch
Epoch: 3/30...  Training Step: 146...  Training loss: 6.6546...  1.4354 sec/batch
Epoch: 3/30...  Training Step: 147...  Training loss: 6.6716...  1.4621 sec/batch
Epoch: 3/30...  Training Step: 148...  Training loss: 6.7012...  1.3936 sec/batch
Epoch: 3/30...  Training Step: 149...  Training loss: 6.6905...  1.5151 sec/batch
Epoch: 3/30...  Training Step: 150...  Training loss: 6.6941...  1.4256 sec/batch
Epoch: 3/30...  Training Step: 151...  Training loss: 6.6855...  1.4370 sec/batch
Epoch: 3/30...  Training Step: 152...  Training loss: 6.6751...  1.4002 sec/batch
Epoch: 3/30...  Training Step: 153...  Training loss: 6.6822...  1.3188 sec/batch
Epoch: 3/30...  Training Step: 154...  Training loss: 6.6631...  1.3952 sec/batch
Epoch: 3/30...  Training Step: 155...  Training loss: 6.6563...  1.4168 sec/batch
Epoch: 3/30...  Training Step: 156...  Training loss: 6.6439...  1.4423 sec/batch
Epoch: 3/30...  Training Step: 157...  Training loss: 6.6463...  1.3898 sec/batch
Epoch: 3/30...  Training Step: 158...  Training loss: 6.6491...  1.4063 sec/batch
Epoch: 3/30...  Training Step: 159...  Training loss: 6.6443...  1.3797 sec/batch
Epoch: 3/30...  Training Step: 160...  Training loss: 6.6465...  1.5161 sec/batch
Epoch: 3/30...  Training Step: 161...  Training loss: 6.6796...  1.4699 sec/batch
Epoch: 3/30...  Training Step: 162...  Training loss: 6.6676...  1.5069 sec/batch
Epoch: 3/30...  Training Step: 163...  Training loss: 6.6813...  1.4635 sec/batch
Epoch: 3/30...  Training Step: 164...  Training loss: 6.6750...  1.3330 sec/batch
Epoch: 3/30...  Training Step: 165...  Training loss: 6.6624...  1.2989 sec/batch
Epoch: 3/30...  Training Step: 166...  Training loss: 6.6769...  1.4570 sec/batch
Epoch: 3/30...  Training Step: 167...  Training loss: 6.6562...  1.4408 sec/batch
Epoch: 3/30...  Training Step: 168...  Training loss: 6.6435...  1.3920 sec/batch
Epoch: 3/30...  Training Step: 169...  Training loss: 6.6517...  1.2854 sec/batch
Epoch: 3/30...  Training Step: 170...  Training loss: 6.6629...  1.5129 sec/batch
Epoch: 3/30...  Training Step: 171...  Training loss: 6.6468...  1.5076 sec/batch
Epoch: 3/30...  Training Step: 172...  Training loss: 6.6569...  1.3418 sec/batch
Epoch: 3/30...  Training Step: 173...  Training loss: 6.6243...  1.3760 sec/batch
Epoch: 3/30...  Training Step: 174...  Training loss: 6.6551...  1.6556 sec/batch
Epoch: 3/30...  Training Step: 175...  Training loss: 6.6432...  1.3339 sec/batch
Epoch: 3/30...  Training Step: 176...  Training loss: 6.6653...  1.4900 sec/batch
Epoch: 3/30...  Training Step: 177...  Training loss: 6.6727...  1.4889 sec/batch
Epoch: 3/30...  Training Step: 178...  Training loss: 6.6330...  1.3389 sec/batch
Epoch: 3/30...  Training Step: 179...  Training loss: 6.6688...  1.4615 sec/batch
Epoch: 3/30...  Training Step: 180...  Training loss: 6.6562...  1.3740 sec/batch
Epoch: 3/30...  Training Step: 181...  Training loss: 6.6506...  1.6786 sec/batch
Epoch: 3/30...  Training Step: 182...  Training loss: 6.6590...  1.3987 sec/batch
Epoch: 3/30...  Training Step: 183...  Training loss: 6.6368...  1.4202 sec/batch
Epoch: 3/30...  Training Step: 184...  Training loss: 6.6504...  1.5768 sec/batch
Epoch: 3/30...  Training Step: 185...  Training loss: 6.6348...  1.4280 sec/batch
Epoch: 3/30...  Training Step: 186...  Training loss: 6.6385...  1.4644 sec/batch
Epoch: 4/30...  Training Step: 187...  Training loss: 6.6399...  1.4111 sec/batch
Epoch: 4/30...  Training Step: 188...  Training loss: 6.6271...  1.5164 sec/batch
Epoch: 4/30...  Training Step: 189...  Training loss: 6.6284...  1.4611 sec/batch
Epoch: 4/30...  Training Step: 190...  Training loss: 6.6636...  1.4363 sec/batch
Epoch: 4/30...  Training Step: 191...  Training loss: 6.6244...  1.5582 sec/batch
Epoch: 4/30...  Training Step: 192...  Training loss: 6.6266...  1.4376 sec/batch
Epoch: 4/30...  Training Step: 193...  Training loss: 6.6167...  1.4376 sec/batch
Epoch: 4/30...  Training Step: 194...  Training loss: 6.6289...  1.4203 sec/batch
Epoch: 4/30...  Training Step: 195...  Training loss: 6.6161...  1.3429 sec/batch
Epoch: 4/30...  Training Step: 196...  Training loss: 6.6173...  1.3890 sec/batch
Epoch: 4/30...  Training Step: 197...  Training loss: 6.6048...  1.5110 sec/batch
Epoch: 4/30...  Training Step: 198...  Training loss: 6.6001...  1.6049 sec/batch
Epoch: 4/30...  Training Step: 199...  Training loss: 6.6237...  1.3756 sec/batch
Epoch: 4/30...  Training Step: 200...  Training loss: 6.6012...  1.4351 sec/batch
Epoch: 4/30...  Training Step: 201...  Training loss: 6.5920...  1.3012 sec/batch
Epoch: 4/30...  Training Step: 202...  Training loss: 6.5909...  1.4274 sec/batch
Epoch: 4/30...  Training Step: 203...  Training loss: 6.7396...  1.3954 sec/batch
Epoch: 4/30...  Training Step: 204...  Training loss: 6.6911...  1.5622 sec/batch
Epoch: 4/30...  Training Step: 205...  Training loss: 6.5948...  1.4575 sec/batch
Epoch: 4/30...  Training Step: 206...  Training loss: 6.6171...  1.4210 sec/batch
Epoch: 4/30...  Training Step: 207...  Training loss: 6.6325...  1.3584 sec/batch
Epoch: 4/30...  Training Step: 208...  Training loss: 6.6389...  1.3613 sec/batch
Epoch: 4/30...  Training Step: 209...  Training loss: 6.6433...  1.3960 sec/batch
Epoch: 4/30...  Training Step: 210...  Training loss: 6.6710...  1.3989 sec/batch
Epoch: 4/30...  Training Step: 211...  Training loss: 6.6611...  1.3411 sec/batch
Epoch: 4/30...  Training Step: 212...  Training loss: 6.6680...  1.4361 sec/batch
Epoch: 4/30...  Training Step: 213...  Training loss: 6.6569...  1.3670 sec/batch
Epoch: 4/30...  Training Step: 214...  Training loss: 6.6459...  1.3343 sec/batch
Epoch: 4/30...  Training Step: 215...  Training loss: 6.6495...  1.3499 sec/batch
Epoch: 4/30...  Training Step: 216...  Training loss: 6.6312...  1.3930 sec/batch
Epoch: 4/30...  Training Step: 217...  Training loss: 6.6189...  1.3544 sec/batch
Epoch: 4/30...  Training Step: 218...  Training loss: 6.6085...  1.4569 sec/batch
Epoch: 4/30...  Training Step: 219...  Training loss: 6.6049...  1.4790 sec/batch
Epoch: 4/30...  Training Step: 220...  Training loss: 6.6037...  1.4657 sec/batch
Epoch: 4/30...  Training Step: 221...  Training loss: 6.5954...  1.3408 sec/batch
Epoch: 4/30...  Training Step: 222...  Training loss: 6.5892...  1.3995 sec/batch
Epoch: 4/30...  Training Step: 223...  Training loss: 6.6093...  1.3170 sec/batch
Epoch: 4/30...  Training Step: 224...  Training loss: 6.5912...  1.3077 sec/batch
Epoch: 4/30...  Training Step: 225...  Training loss: 6.5940...  1.3513 sec/batch
Epoch: 4/30...  Training Step: 226...  Training loss: 6.5731...  1.4222 sec/batch
Epoch: 4/30...  Training Step: 227...  Training loss: 6.5405...  1.2691 sec/batch
Epoch: 4/30...  Training Step: 228...  Training loss: 6.5799...  1.5480 sec/batch
Epoch: 4/30...  Training Step: 229...  Training loss: 7.0135...  1.4866 sec/batch
Epoch: 4/30...  Training Step: 230...  Training loss: 6.4949...  1.3864 sec/batch
Epoch: 4/30...  Training Step: 231...  Training loss: 6.6016...  1.3690 sec/batch
Epoch: 4/30...  Training Step: 232...  Training loss: 6.5565...  1.3309 sec/batch
Epoch: 4/30...  Training Step: 233...  Training loss: 6.5071...  1.3420 sec/batch
Epoch: 4/30...  Training Step: 234...  Training loss: 6.5206...  1.3662 sec/batch
Epoch: 4/30...  Training Step: 235...  Training loss: 6.4496...  1.6192 sec/batch
Epoch: 4/30...  Training Step: 236...  Training loss: 6.5612...  1.4943 sec/batch
Epoch: 4/30...  Training Step: 237...  Training loss: 6.5442...  1.3444 sec/batch
Epoch: 4/30...  Training Step: 238...  Training loss: 6.5884...  1.6131 sec/batch
Epoch: 4/30...  Training Step: 239...  Training loss: 6.5637...  1.4195 sec/batch
Epoch: 4/30...  Training Step: 240...  Training loss: 6.5295...  1.4300 sec/batch
Epoch: 4/30...  Training Step: 241...  Training loss: 6.5340...  1.4594 sec/batch
Epoch: 4/30...  Training Step: 242...  Training loss: 6.5311...  1.3751 sec/batch
Epoch: 4/30...  Training Step: 243...  Training loss: 6.4891...  1.4983 sec/batch
Epoch: 4/30...  Training Step: 244...  Training loss: 6.4935...  1.3653 sec/batch
Epoch: 4/30...  Training Step: 245...  Training loss: 6.4240...  1.3761 sec/batch
Epoch: 4/30...  Training Step: 246...  Training loss: 6.4533...  1.4847 sec/batch
Epoch: 4/30...  Training Step: 247...  Training loss: 6.4556...  1.5116 sec/batch
Epoch: 4/30...  Training Step: 248...  Training loss: 6.4158...  1.3936 sec/batch
Epoch: 5/30...  Training Step: 249...  Training loss: 6.4344...  1.6398 sec/batch
Epoch: 5/30...  Training Step: 250...  Training loss: 6.4764...  1.5166 sec/batch
Epoch: 5/30...  Training Step: 251...  Training loss: 6.4327...  1.4664 sec/batch
Epoch: 5/30...  Training Step: 252...  Training loss: 6.4798...  1.4800 sec/batch
Epoch: 5/30...  Training Step: 253...  Training loss: 6.4417...  1.4594 sec/batch
Epoch: 5/30...  Training Step: 254...  Training loss: 6.4356...  1.4627 sec/batch
Epoch: 5/30...  Training Step: 255...  Training loss: 6.4349...  1.3812 sec/batch
Epoch: 5/30...  Training Step: 256...  Training loss: 6.4491...  1.3858 sec/batch
Epoch: 5/30...  Training Step: 257...  Training loss: 6.4177...  1.5806 sec/batch
Epoch: 5/30...  Training Step: 258...  Training loss: 6.4282...  1.5258 sec/batch
Epoch: 5/30...  Training Step: 259...  Training loss: 6.4050...  1.5720 sec/batch
Epoch: 5/30...  Training Step: 260...  Training loss: 6.3922...  1.5523 sec/batch
Epoch: 5/30...  Training Step: 261...  Training loss: 6.4115...  1.4733 sec/batch
Epoch: 5/30...  Training Step: 262...  Training loss: 6.3765...  1.5569 sec/batch
Epoch: 5/30...  Training Step: 263...  Training loss: 6.3718...  1.6843 sec/batch
Epoch: 5/30...  Training Step: 264...  Training loss: 6.3387...  1.4579 sec/batch
Epoch: 5/30...  Training Step: 265...  Training loss: 6.3532...  1.4926 sec/batch
Epoch: 5/30...  Training Step: 266...  Training loss: 6.3404...  1.5653 sec/batch
Epoch: 5/30...  Training Step: 267...  Training loss: 6.2864...  1.4848 sec/batch
Epoch: 5/30...  Training Step: 268...  Training loss: 6.2848...  1.4542 sec/batch
Epoch: 5/30...  Training Step: 269...  Training loss: 6.2881...  1.5501 sec/batch
Epoch: 5/30...  Training Step: 270...  Training loss: 6.2913...  1.7591 sec/batch
Epoch: 5/30...  Training Step: 271...  Training loss: 6.2944...  1.5260 sec/batch
Epoch: 5/30...  Training Step: 272...  Training loss: 6.3272...  1.3465 sec/batch
Epoch: 5/30...  Training Step: 273...  Training loss: 6.3252...  1.5885 sec/batch
Epoch: 5/30...  Training Step: 274...  Training loss: 6.3593...  1.4709 sec/batch
Epoch: 5/30...  Training Step: 275...  Training loss: 6.3083...  1.4585 sec/batch
Epoch: 5/30...  Training Step: 276...  Training loss: 6.2927...  1.5333 sec/batch
Epoch: 5/30...  Training Step: 277...  Training loss: 6.2938...  1.4861 sec/batch
Epoch: 5/30...  Training Step: 278...  Training loss: 6.2585...  1.4529 sec/batch
Epoch: 5/30...  Training Step: 279...  Training loss: 6.2578...  1.3914 sec/batch
Epoch: 5/30...  Training Step: 280...  Training loss: 6.2234...  1.5085 sec/batch
Epoch: 5/30...  Training Step: 281...  Training loss: 6.2260...  1.5259 sec/batch
Epoch: 5/30...  Training Step: 282...  Training loss: 6.2204...  1.4164 sec/batch
Epoch: 5/30...  Training Step: 283...  Training loss: 6.2027...  1.4955 sec/batch
Epoch: 5/30...  Training Step: 284...  Training loss: 6.2071...  1.3983 sec/batch
Epoch: 5/30...  Training Step: 285...  Training loss: 6.2170...  1.4101 sec/batch
Epoch: 5/30...  Training Step: 286...  Training loss: 6.2157...  1.4290 sec/batch
Epoch: 5/30...  Training Step: 287...  Training loss: 6.2482...  1.6261 sec/batch
Epoch: 5/30...  Training Step: 288...  Training loss: 6.2134...  1.5056 sec/batch
Epoch: 5/30...  Training Step: 289...  Training loss: 6.2013...  1.4428 sec/batch
Epoch: 5/30...  Training Step: 290...  Training loss: 6.2019...  1.5632 sec/batch
Epoch: 5/30...  Training Step: 291...  Training loss: 6.1678...  1.4889 sec/batch
Epoch: 5/30...  Training Step: 292...  Training loss: 6.1505...  1.5197 sec/batch
Epoch: 5/30...  Training Step: 293...  Training loss: 6.1431...  1.4593 sec/batch
Epoch: 5/30...  Training Step: 294...  Training loss: 6.1472...  1.3339 sec/batch
Epoch: 5/30...  Training Step: 295...  Training loss: 6.1270...  1.4889 sec/batch
Epoch: 5/30...  Training Step: 296...  Training loss: 6.1338...  1.4490 sec/batch
Epoch: 5/30...  Training Step: 297...  Training loss: 6.0926...  1.5077 sec/batch
Epoch: 5/30...  Training Step: 298...  Training loss: 6.1220...  1.5050 sec/batch
Epoch: 5/30...  Training Step: 299...  Training loss: 6.0990...  1.3943 sec/batch
Epoch: 5/30...  Training Step: 300...  Training loss: 6.1205...  1.4848 sec/batch
Epoch: 5/30...  Training Step: 301...  Training loss: 6.1337...  1.3461 sec/batch
Epoch: 5/30...  Training Step: 302...  Training loss: 6.0855...  1.3493 sec/batch
Epoch: 5/30...  Training Step: 303...  Training loss: 6.1190...  1.3921 sec/batch
Epoch: 5/30...  Training Step: 304...  Training loss: 6.1040...  1.4300 sec/batch
Epoch: 5/30...  Training Step: 305...  Training loss: 6.0943...  1.4806 sec/batch
Epoch: 5/30...  Training Step: 306...  Training loss: 6.1026...  1.3832 sec/batch
Epoch: 5/30...  Training Step: 307...  Training loss: 6.0759...  1.4078 sec/batch
Epoch: 5/30...  Training Step: 308...  Training loss: 6.0855...  1.5088 sec/batch
Epoch: 5/30...  Training Step: 309...  Training loss: 6.0711...  1.4256 sec/batch
Epoch: 5/30...  Training Step: 310...  Training loss: 6.0623...  1.4104 sec/batch
Epoch: 6/30...  Training Step: 311...  Training loss: 6.2652...  1.3217 sec/batch
Epoch: 6/30...  Training Step: 312...  Training loss: 6.0779...  1.3885 sec/batch
Epoch: 6/30...  Training Step: 313...  Training loss: 6.0600...  1.3100 sec/batch
Epoch: 6/30...  Training Step: 314...  Training loss: 6.1161...  1.5049 sec/batch
Epoch: 6/30...  Training Step: 315...  Training loss: 6.0674...  1.5620 sec/batch
Epoch: 6/30...  Training Step: 316...  Training loss: 6.0858...  1.3971 sec/batch
Epoch: 6/30...  Training Step: 317...  Training loss: 6.0583...  1.4239 sec/batch
Epoch: 6/30...  Training Step: 318...  Training loss: 6.0860...  1.2929 sec/batch
Epoch: 6/30...  Training Step: 319...  Training loss: 6.0656...  1.5431 sec/batch
Epoch: 6/30...  Training Step: 320...  Training loss: 6.0598...  1.3905 sec/batch
Epoch: 6/30...  Training Step: 321...  Training loss: 6.0601...  1.4114 sec/batch
Epoch: 6/30...  Training Step: 322...  Training loss: 6.0527...  1.3591 sec/batch
Epoch: 6/30...  Training Step: 323...  Training loss: 6.0812...  1.4910 sec/batch
Epoch: 6/30...  Training Step: 324...  Training loss: 6.0655...  1.4707 sec/batch
Epoch: 6/30...  Training Step: 325...  Training loss: 6.0519...  1.4192 sec/batch
Epoch: 6/30...  Training Step: 326...  Training loss: 6.0489...  1.4573 sec/batch
Epoch: 6/30...  Training Step: 327...  Training loss: 6.0583...  1.4448 sec/batch
Epoch: 6/30...  Training Step: 328...  Training loss: 6.0533...  1.3198 sec/batch
Epoch: 6/30...  Training Step: 329...  Training loss: 6.0128...  1.4048 sec/batch
Epoch: 6/30...  Training Step: 330...  Training loss: 6.0152...  1.4718 sec/batch
Epoch: 6/30...  Training Step: 331...  Training loss: 6.0194...  1.4675 sec/batch
Epoch: 6/30...  Training Step: 332...  Training loss: 6.0318...  1.3637 sec/batch
Epoch: 6/30...  Training Step: 333...  Training loss: 6.0438...  1.2739 sec/batch
Epoch: 6/30...  Training Step: 334...  Training loss: 6.0769...  1.3904 sec/batch
Epoch: 6/30...  Training Step: 335...  Training loss: 6.0762...  1.3771 sec/batch
Epoch: 6/30...  Training Step: 336...  Training loss: 6.0804...  1.4563 sec/batch
Epoch: 6/30...  Training Step: 337...  Training loss: 6.0660...  1.5034 sec/batch
Epoch: 6/30...  Training Step: 338...  Training loss: 6.0545...  1.4297 sec/batch
Epoch: 6/30...  Training Step: 339...  Training loss: 6.0530...  1.4397 sec/batch
Epoch: 6/30...  Training Step: 340...  Training loss: 6.0336...  1.4297 sec/batch
Epoch: 6/30...  Training Step: 341...  Training loss: 6.0276...  1.3882 sec/batch
Epoch: 6/30...  Training Step: 342...  Training loss: 6.0105...  1.3652 sec/batch
Epoch: 6/30...  Training Step: 343...  Training loss: 6.0100...  1.3420 sec/batch
Epoch: 6/30...  Training Step: 344...  Training loss: 6.0095...  1.4337 sec/batch
Epoch: 6/30...  Training Step: 345...  Training loss: 5.9977...  1.5364 sec/batch
Epoch: 6/30...  Training Step: 346...  Training loss: 5.9994...  1.4502 sec/batch
Epoch: 6/30...  Training Step: 347...  Training loss: 6.0265...  1.2895 sec/batch
Epoch: 6/30...  Training Step: 348...  Training loss: 6.0237...  1.3289 sec/batch
Epoch: 6/30...  Training Step: 349...  Training loss: 6.0368...  1.4818 sec/batch
Epoch: 6/30...  Training Step: 350...  Training loss: 6.0417...  1.4780 sec/batch
Epoch: 6/30...  Training Step: 351...  Training loss: 6.0181...  1.4928 sec/batch
Epoch: 6/30...  Training Step: 352...  Training loss: 6.0382...  1.4458 sec/batch
Epoch: 6/30...  Training Step: 353...  Training loss: 6.0150...  1.4452 sec/batch
Epoch: 6/30...  Training Step: 354...  Training loss: 6.0023...  1.3255 sec/batch
Epoch: 6/30...  Training Step: 355...  Training loss: 6.0081...  1.4280 sec/batch
Epoch: 6/30...  Training Step: 356...  Training loss: 6.0207...  1.6268 sec/batch
Epoch: 6/30...  Training Step: 357...  Training loss: 6.0021...  1.6462 sec/batch
Epoch: 6/30...  Training Step: 358...  Training loss: 6.0131...  1.3841 sec/batch
Epoch: 6/30...  Training Step: 359...  Training loss: 5.9779...  1.4416 sec/batch
Epoch: 6/30...  Training Step: 360...  Training loss: 6.0090...  1.4911 sec/batch
Epoch: 6/30...  Training Step: 361...  Training loss: 5.9960...  1.3617 sec/batch
Epoch: 6/30...  Training Step: 362...  Training loss: 6.0132...  1.2711 sec/batch
Epoch: 6/30...  Training Step: 363...  Training loss: 6.0187...  1.5614 sec/batch
Epoch: 6/30...  Training Step: 364...  Training loss: 5.9786...  1.5104 sec/batch
Epoch: 6/30...  Training Step: 365...  Training loss: 6.0130...  1.4833 sec/batch
Epoch: 6/30...  Training Step: 366...  Training loss: 5.9961...  1.3896 sec/batch
Epoch: 6/30...  Training Step: 367...  Training loss: 5.9908...  1.3587 sec/batch
Epoch: 6/30...  Training Step: 368...  Training loss: 6.0039...  1.5506 sec/batch
Epoch: 6/30...  Training Step: 369...  Training loss: 5.9763...  1.3743 sec/batch
Epoch: 6/30...  Training Step: 370...  Training loss: 5.9906...  1.3942 sec/batch
Epoch: 6/30...  Training Step: 371...  Training loss: 5.9761...  1.3883 sec/batch
Epoch: 6/30...  Training Step: 372...  Training loss: 5.9675...  1.2744 sec/batch
Epoch: 7/30...  Training Step: 373...  Training loss: 6.3236...  1.4967 sec/batch
Epoch: 7/30...  Training Step: 374...  Training loss: 6.2000...  1.4099 sec/batch
Epoch: 7/30...  Training Step: 375...  Training loss: 6.1754...  1.4600 sec/batch
Epoch: 7/30...  Training Step: 376...  Training loss: 6.1957...  1.4045 sec/batch
Epoch: 7/30...  Training Step: 377...  Training loss: 6.1783...  1.3163 sec/batch
Epoch: 7/30...  Training Step: 378...  Training loss: 6.1593...  1.4458 sec/batch
Epoch: 7/30...  Training Step: 379...  Training loss: 6.1498...  1.2658 sec/batch
Epoch: 7/30...  Training Step: 380...  Training loss: 6.1682...  1.3523 sec/batch
Epoch: 7/30...  Training Step: 381...  Training loss: 6.1574...  1.4220 sec/batch
Epoch: 7/30...  Training Step: 382...  Training loss: 6.1515...  1.4621 sec/batch
Epoch: 7/30...  Training Step: 383...  Training loss: 6.1366...  1.2920 sec/batch
Epoch: 7/30...  Training Step: 384...  Training loss: 6.1337...  1.4412 sec/batch
Epoch: 7/30...  Training Step: 385...  Training loss: 6.1654...  1.4866 sec/batch
Epoch: 7/30...  Training Step: 386...  Training loss: 6.1496...  1.4578 sec/batch
Epoch: 7/30...  Training Step: 387...  Training loss: 6.1406...  1.3922 sec/batch
Epoch: 7/30...  Training Step: 388...  Training loss: 6.1369...  1.4554 sec/batch
Epoch: 7/30...  Training Step: 389...  Training loss: 6.1432...  1.3145 sec/batch
Epoch: 7/30...  Training Step: 390...  Training loss: 6.0097...  1.3281 sec/batch
Epoch: 7/30...  Training Step: 391...  Training loss: 5.9363...  1.4797 sec/batch
Epoch: 7/30...  Training Step: 392...  Training loss: 5.9426...  1.5000 sec/batch
Epoch: 7/30...  Training Step: 393...  Training loss: 5.9529...  1.3895 sec/batch
Epoch: 7/30...  Training Step: 394...  Training loss: 5.9606...  1.4967 sec/batch
Epoch: 7/30...  Training Step: 395...  Training loss: 5.9660...  1.4813 sec/batch
Epoch: 7/30...  Training Step: 396...  Training loss: 5.9962...  1.4103 sec/batch
Epoch: 7/30...  Training Step: 397...  Training loss: 6.0023...  1.2705 sec/batch
Epoch: 7/30...  Training Step: 398...  Training loss: 6.0007...  1.3857 sec/batch
Epoch: 7/30...  Training Step: 399...  Training loss: 5.9847...  1.3141 sec/batch
Epoch: 7/30...  Training Step: 400...  Training loss: 5.9792...  1.3262 sec/batch
Epoch: 7/30...  Training Step: 401...  Training loss: 5.9771...  1.4670 sec/batch
Epoch: 7/30...  Training Step: 402...  Training loss: 5.9594...  1.4972 sec/batch
Epoch: 7/30...  Training Step: 403...  Training loss: 5.9481...  1.3799 sec/batch
Epoch: 7/30...  Training Step: 404...  Training loss: 5.9342...  1.3300 sec/batch
Epoch: 7/30...  Training Step: 405...  Training loss: 5.9326...  1.3835 sec/batch
Epoch: 7/30...  Training Step: 406...  Training loss: 5.9359...  1.4415 sec/batch
Epoch: 7/30...  Training Step: 407...  Training loss: 5.9236...  1.4491 sec/batch
Epoch: 7/30...  Training Step: 408...  Training loss: 5.9295...  1.3330 sec/batch
Epoch: 7/30...  Training Step: 409...  Training loss: 6.9615...  1.4359 sec/batch
Epoch: 7/30...  Training Step: 410...  Training loss: 7.1233...  1.2893 sec/batch
Epoch: 7/30...  Training Step: 411...  Training loss: 6.6494...  1.3922 sec/batch
Epoch: 7/30...  Training Step: 412...  Training loss: 6.4271...  1.3717 sec/batch
Epoch: 7/30...  Training Step: 413...  Training loss: 6.3131...  1.3145 sec/batch
Epoch: 7/30...  Training Step: 414...  Training loss: 6.3083...  1.3671 sec/batch
Epoch: 7/30...  Training Step: 415...  Training loss: 6.2881...  1.4529 sec/batch
Epoch: 7/30...  Training Step: 416...  Training loss: 6.2712...  1.3988 sec/batch
Epoch: 7/30...  Training Step: 417...  Training loss: 6.2331...  1.3179 sec/batch
Epoch: 7/30...  Training Step: 418...  Training loss: 6.2038...  1.3190 sec/batch
Epoch: 7/30...  Training Step: 419...  Training loss: 6.1943...  1.3728 sec/batch
Epoch: 7/30...  Training Step: 420...  Training loss: 6.1787...  1.3975 sec/batch
Epoch: 7/30...  Training Step: 421...  Training loss: 6.1378...  1.5136 sec/batch
Epoch: 7/30...  Training Step: 422...  Training loss: 6.1591...  1.5034 sec/batch
Epoch: 7/30...  Training Step: 423...  Training loss: 6.1322...  1.5919 sec/batch
Epoch: 7/30...  Training Step: 424...  Training loss: 6.1571...  1.4972 sec/batch
Epoch: 7/30...  Training Step: 425...  Training loss: 6.1459...  1.2994 sec/batch
Epoch: 7/30...  Training Step: 426...  Training loss: 6.1025...  1.4996 sec/batch
Epoch: 7/30...  Training Step: 427...  Training loss: 6.1142...  1.4690 sec/batch
Epoch: 7/30...  Training Step: 428...  Training loss: 6.0864...  1.5461 sec/batch
Epoch: 7/30...  Training Step: 429...  Training loss: 6.0749...  1.6125 sec/batch
Epoch: 7/30...  Training Step: 430...  Training loss: 6.0869...  1.5730 sec/batch
Epoch: 7/30...  Training Step: 431...  Training loss: 6.0406...  1.4557 sec/batch
Epoch: 7/30...  Training Step: 432...  Training loss: 6.0451...  1.5755 sec/batch
Epoch: 7/30...  Training Step: 433...  Training loss: 6.0335...  1.5847 sec/batch
Epoch: 7/30...  Training Step: 434...  Training loss: 6.0161...  1.5226 sec/batch
Epoch: 8/30...  Training Step: 435...  Training loss: 6.1730...  1.4008 sec/batch
Epoch: 8/30...  Training Step: 436...  Training loss: 6.0045...  1.5346 sec/batch
Epoch: 8/30...  Training Step: 437...  Training loss: 6.0038...  1.4193 sec/batch
Epoch: 8/30...  Training Step: 438...  Training loss: 6.0384...  1.5030 sec/batch
Epoch: 8/30...  Training Step: 439...  Training loss: 5.9995...  1.3949 sec/batch
Epoch: 8/30...  Training Step: 440...  Training loss: 5.9920...  1.3817 sec/batch
Epoch: 8/30...  Training Step: 441...  Training loss: 5.9864...  1.3540 sec/batch
Epoch: 8/30...  Training Step: 442...  Training loss: 5.9958...  1.3384 sec/batch
Epoch: 8/30...  Training Step: 443...  Training loss: 5.9847...  1.4998 sec/batch
Epoch: 8/30...  Training Step: 444...  Training loss: 5.9809...  1.3713 sec/batch
Epoch: 8/30...  Training Step: 445...  Training loss: 5.9691...  1.3778 sec/batch
Epoch: 8/30...  Training Step: 446...  Training loss: 5.9657...  1.4015 sec/batch
Epoch: 8/30...  Training Step: 447...  Training loss: 5.9907...  1.3755 sec/batch
Epoch: 8/30...  Training Step: 448...  Training loss: 5.9686...  1.4007 sec/batch
Epoch: 8/30...  Training Step: 449...  Training loss: 5.9597...  1.3834 sec/batch
Epoch: 8/30...  Training Step: 450...  Training loss: 5.9559...  1.3828 sec/batch
Epoch: 8/30...  Training Step: 451...  Training loss: 5.9641...  1.5718 sec/batch
Epoch: 8/30...  Training Step: 452...  Training loss: 5.9583...  1.5138 sec/batch
Epoch: 8/30...  Training Step: 453...  Training loss: 5.9239...  1.3631 sec/batch
Epoch: 8/30...  Training Step: 454...  Training loss: 5.9232...  1.3675 sec/batch
Epoch: 8/30...  Training Step: 455...  Training loss: 5.9304...  1.4387 sec/batch
Epoch: 8/30...  Training Step: 456...  Training loss: 5.9413...  1.3562 sec/batch
Epoch: 8/30...  Training Step: 457...  Training loss: 5.9448...  1.3195 sec/batch
Epoch: 8/30...  Training Step: 458...  Training loss: 5.9804...  1.3889 sec/batch
Epoch: 8/30...  Training Step: 459...  Training loss: 5.9839...  1.3419 sec/batch
Epoch: 8/30...  Training Step: 460...  Training loss: 5.9856...  1.4120 sec/batch
Epoch: 8/30...  Training Step: 461...  Training loss: 5.9699...  1.3960 sec/batch
Epoch: 8/30...  Training Step: 462...  Training loss: 5.9632...  1.4453 sec/batch
Epoch: 8/30...  Training Step: 463...  Training loss: 5.9626...  1.4301 sec/batch
Epoch: 8/30...  Training Step: 464...  Training loss: 5.9418...  1.5507 sec/batch
Epoch: 8/30...  Training Step: 465...  Training loss: 5.9392...  1.3822 sec/batch
Epoch: 8/30...  Training Step: 466...  Training loss: 5.9208...  1.4108 sec/batch
Epoch: 8/30...  Training Step: 467...  Training loss: 5.9256...  1.3855 sec/batch
Epoch: 8/30...  Training Step: 468...  Training loss: 5.9252...  1.4626 sec/batch
Epoch: 8/30...  Training Step: 469...  Training loss: 5.9212...  1.3768 sec/batch
Epoch: 8/30...  Training Step: 470...  Training loss: 5.9327...  1.3436 sec/batch
Epoch: 8/30...  Training Step: 471...  Training loss: 5.9673...  1.3705 sec/batch
Epoch: 8/30...  Training Step: 472...  Training loss: 5.9579...  1.5082 sec/batch
Epoch: 8/30...  Training Step: 473...  Training loss: 5.9685...  1.4142 sec/batch
Epoch: 8/30...  Training Step: 474...  Training loss: 5.9640...  1.6172 sec/batch
Epoch: 8/30...  Training Step: 475...  Training loss: 5.9381...  1.4181 sec/batch
Epoch: 8/30...  Training Step: 476...  Training loss: 5.9564...  1.4680 sec/batch
Epoch: 8/30...  Training Step: 477...  Training loss: 5.9416...  1.2676 sec/batch
Epoch: 8/30...  Training Step: 478...  Training loss: 5.9186...  1.4568 sec/batch
Epoch: 8/30...  Training Step: 479...  Training loss: 5.9322...  1.6252 sec/batch
Epoch: 8/30...  Training Step: 480...  Training loss: 5.9388...  1.3922 sec/batch
Epoch: 8/30...  Training Step: 481...  Training loss: 5.9311...  1.4052 sec/batch
Epoch: 8/30...  Training Step: 482...  Training loss: 5.9366...  1.4476 sec/batch
Epoch: 8/30...  Training Step: 483...  Training loss: 5.9011...  1.5639 sec/batch
Epoch: 8/30...  Training Step: 484...  Training loss: 5.9287...  1.3013 sec/batch
Epoch: 8/30...  Training Step: 485...  Training loss: 5.9203...  1.2940 sec/batch
Epoch: 8/30...  Training Step: 486...  Training loss: 5.9354...  1.3785 sec/batch
Epoch: 8/30...  Training Step: 487...  Training loss: 5.9504...  1.3897 sec/batch
Epoch: 8/30...  Training Step: 488...  Training loss: 5.9078...  1.2569 sec/batch
Epoch: 8/30...  Training Step: 489...  Training loss: 5.9417...  1.3325 sec/batch
Epoch: 8/30...  Training Step: 490...  Training loss: 5.9242...  1.4932 sec/batch
Epoch: 8/30...  Training Step: 491...  Training loss: 5.9212...  1.3072 sec/batch
Epoch: 8/30...  Training Step: 492...  Training loss: 5.9296...  1.4524 sec/batch
Epoch: 8/30...  Training Step: 493...  Training loss: 5.9021...  1.4090 sec/batch
Epoch: 8/30...  Training Step: 494...  Training loss: 5.9165...  1.3387 sec/batch
Epoch: 8/30...  Training Step: 495...  Training loss: 5.9099...  1.4231 sec/batch
Epoch: 8/30...  Training Step: 496...  Training loss: 5.8926...  1.2801 sec/batch
Epoch: 9/30...  Training Step: 497...  Training loss: 6.0572...  1.5259 sec/batch
Epoch: 9/30...  Training Step: 498...  Training loss: 5.8958...  1.4596 sec/batch
Epoch: 9/30...  Training Step: 499...  Training loss: 5.8943...  1.2605 sec/batch
Epoch: 9/30...  Training Step: 500...  Training loss: 5.9363...  1.6336 sec/batch
Epoch: 9/30...  Training Step: 501...  Training loss: 5.9070...  1.5013 sec/batch
Epoch: 9/30...  Training Step: 502...  Training loss: 5.9050...  1.4271 sec/batch
Epoch: 9/30...  Training Step: 503...  Training loss: 5.9017...  1.4907 sec/batch
Epoch: 9/30...  Training Step: 504...  Training loss: 5.9148...  1.3943 sec/batch
Epoch: 9/30...  Training Step: 505...  Training loss: 5.9006...  1.4504 sec/batch
Epoch: 9/30...  Training Step: 506...  Training loss: 5.9008...  1.3668 sec/batch
Epoch: 9/30...  Training Step: 507...  Training loss: 5.8911...  1.3291 sec/batch
Epoch: 9/30...  Training Step: 508...  Training loss: 5.8887...  1.4686 sec/batch
Epoch: 9/30...  Training Step: 509...  Training loss: 5.9163...  1.4532 sec/batch
Epoch: 9/30...  Training Step: 510...  Training loss: 5.9005...  1.4128 sec/batch
Epoch: 9/30...  Training Step: 511...  Training loss: 5.8960...  1.2804 sec/batch
Epoch: 9/30...  Training Step: 512...  Training loss: 5.8886...  1.3299 sec/batch
Epoch: 9/30...  Training Step: 513...  Training loss: 5.8973...  1.3843 sec/batch
Epoch: 9/30...  Training Step: 514...  Training loss: 5.8902...  1.4243 sec/batch
Epoch: 9/30...  Training Step: 515...  Training loss: 5.8551...  1.2866 sec/batch
Epoch: 9/30...  Training Step: 516...  Training loss: 5.8649...  1.5793 sec/batch
Epoch: 9/30...  Training Step: 517...  Training loss: 5.8704...  1.3444 sec/batch
Epoch: 9/30...  Training Step: 518...  Training loss: 5.8822...  1.3565 sec/batch
Epoch: 9/30...  Training Step: 519...  Training loss: 5.8869...  1.4102 sec/batch
Epoch: 9/30...  Training Step: 520...  Training loss: 5.9198...  1.3315 sec/batch
Epoch: 9/30...  Training Step: 521...  Training loss: 5.9263...  1.4518 sec/batch
Epoch: 9/30...  Training Step: 522...  Training loss: 5.9256...  1.4869 sec/batch
Epoch: 9/30...  Training Step: 523...  Training loss: 5.9127...  1.3718 sec/batch
Epoch: 9/30...  Training Step: 524...  Training loss: 5.9037...  1.4231 sec/batch
Epoch: 9/30...  Training Step: 525...  Training loss: 5.8987...  1.3675 sec/batch
Epoch: 9/30...  Training Step: 526...  Training loss: 5.8785...  1.4709 sec/batch
Epoch: 9/30...  Training Step: 527...  Training loss: 5.8784...  1.3370 sec/batch
Epoch: 9/30...  Training Step: 528...  Training loss: 5.8577...  1.5904 sec/batch
Epoch: 9/30...  Training Step: 529...  Training loss: 5.8614...  1.4046 sec/batch
Epoch: 9/30...  Training Step: 530...  Training loss: 5.8591...  1.4087 sec/batch
Epoch: 9/30...  Training Step: 531...  Training loss: 5.8434...  1.3508 sec/batch
Epoch: 9/30...  Training Step: 532...  Training loss: 5.8561...  1.5100 sec/batch
Epoch: 9/30...  Training Step: 533...  Training loss: 5.8894...  1.3805 sec/batch
Epoch: 9/30...  Training Step: 534...  Training loss: 5.8818...  1.4237 sec/batch
Epoch: 9/30...  Training Step: 535...  Training loss: 5.8908...  1.5228 sec/batch
Epoch: 9/30...  Training Step: 536...  Training loss: 5.8958...  1.3909 sec/batch
Epoch: 9/30...  Training Step: 537...  Training loss: 5.8711...  1.3624 sec/batch
Epoch: 9/30...  Training Step: 538...  Training loss: 5.8805...  1.3136 sec/batch
Epoch: 9/30...  Training Step: 539...  Training loss: 5.8731...  1.3593 sec/batch
Epoch: 9/30...  Training Step: 540...  Training loss: 5.8507...  1.4919 sec/batch
Epoch: 9/30...  Training Step: 541...  Training loss: 5.8604...  1.5010 sec/batch
Epoch: 9/30...  Training Step: 542...  Training loss: 5.8785...  1.4552 sec/batch
Epoch: 9/30...  Training Step: 543...  Training loss: 5.8560...  1.4391 sec/batch
Epoch: 9/30...  Training Step: 544...  Training loss: 5.8704...  1.4129 sec/batch
Epoch: 9/30...  Training Step: 545...  Training loss: 5.8244...  1.3864 sec/batch
Epoch: 9/30...  Training Step: 546...  Training loss: 5.8711...  1.4684 sec/batch
Epoch: 9/30...  Training Step: 547...  Training loss: 5.8454...  1.3587 sec/batch
Epoch: 9/30...  Training Step: 548...  Training loss: 5.8586...  1.4265 sec/batch
Epoch: 9/30...  Training Step: 549...  Training loss: 5.8747...  1.4017 sec/batch
Epoch: 9/30...  Training Step: 550...  Training loss: 5.8328...  1.3309 sec/batch
Epoch: 9/30...  Training Step: 551...  Training loss: 5.8658...  1.4693 sec/batch
Epoch: 9/30...  Training Step: 552...  Training loss: 5.8469...  1.3176 sec/batch
Epoch: 9/30...  Training Step: 553...  Training loss: 5.8551...  1.4927 sec/batch
Epoch: 9/30...  Training Step: 554...  Training loss: 5.8638...  1.4146 sec/batch
Epoch: 9/30...  Training Step: 555...  Training loss: 5.8293...  1.4592 sec/batch
Epoch: 9/30...  Training Step: 556...  Training loss: 5.8511...  1.3262 sec/batch
Epoch: 9/30...  Training Step: 557...  Training loss: 5.8402...  1.3179 sec/batch
Epoch: 9/30...  Training Step: 558...  Training loss: 5.8160...  1.3557 sec/batch
Epoch: 10/30...  Training Step: 559...  Training loss: 5.9625...  1.3486 sec/batch
Epoch: 10/30...  Training Step: 560...  Training loss: 5.8125...  1.3973 sec/batch
Epoch: 10/30...  Training Step: 561...  Training loss: 5.8072...  1.4244 sec/batch
Epoch: 10/30...  Training Step: 562...  Training loss: 5.8483...  1.4336 sec/batch
Epoch: 10/30...  Training Step: 563...  Training loss: 5.8143...  1.4702 sec/batch
Epoch: 10/30...  Training Step: 564...  Training loss: 5.8206...  1.5021 sec/batch
Epoch: 10/30...  Training Step: 565...  Training loss: 5.8074...  1.3942 sec/batch
Epoch: 10/30...  Training Step: 566...  Training loss: 5.8269...  1.4537 sec/batch
Epoch: 10/30...  Training Step: 567...  Training loss: 5.8040...  1.3219 sec/batch
Epoch: 10/30...  Training Step: 568...  Training loss: 5.8129...  1.4239 sec/batch
Epoch: 10/30...  Training Step: 569...  Training loss: 5.7938...  1.2874 sec/batch
Epoch: 10/30...  Training Step: 570...  Training loss: 5.7947...  1.3017 sec/batch
Epoch: 10/30...  Training Step: 571...  Training loss: 5.8238...  1.4393 sec/batch
Epoch: 10/30...  Training Step: 572...  Training loss: 5.7986...  1.4593 sec/batch
Epoch: 10/30...  Training Step: 573...  Training loss: 5.7886...  1.4216 sec/batch
Epoch: 10/30...  Training Step: 574...  Training loss: 5.7850...  1.4434 sec/batch
Epoch: 10/30...  Training Step: 575...  Training loss: 5.7831...  1.4654 sec/batch
Epoch: 10/30...  Training Step: 576...  Training loss: 5.7900...  1.4326 sec/batch
Epoch: 10/30...  Training Step: 577...  Training loss: 5.7411...  1.3150 sec/batch
Epoch: 10/30...  Training Step: 578...  Training loss: 5.7605...  1.6420 sec/batch
Epoch: 10/30...  Training Step: 579...  Training loss: 5.7527...  1.2896 sec/batch
Epoch: 10/30...  Training Step: 580...  Training loss: 5.7717...  1.2482 sec/batch
Epoch: 10/30...  Training Step: 581...  Training loss: 5.7702...  1.5909 sec/batch
Epoch: 10/30...  Training Step: 582...  Training loss: 5.8069...  1.3923 sec/batch
Epoch: 10/30...  Training Step: 583...  Training loss: 5.8126...  1.4027 sec/batch
Epoch: 10/30...  Training Step: 584...  Training loss: 5.8125...  1.4149 sec/batch
Epoch: 10/30...  Training Step: 585...  Training loss: 5.7863...  1.3454 sec/batch
Epoch: 10/30...  Training Step: 586...  Training loss: 5.7892...  1.5742 sec/batch
Epoch: 10/30...  Training Step: 587...  Training loss: 5.7665...  1.3460 sec/batch
Epoch: 10/30...  Training Step: 588...  Training loss: 5.7688...  1.3245 sec/batch
Epoch: 10/30...  Training Step: 589...  Training loss: 5.7557...  1.2977 sec/batch
Epoch: 10/30...  Training Step: 590...  Training loss: 5.7308...  1.4666 sec/batch
Epoch: 10/30...  Training Step: 591...  Training loss: 5.7337...  1.3698 sec/batch
Epoch: 10/30...  Training Step: 592...  Training loss: 5.7247...  1.4808 sec/batch
Epoch: 10/30...  Training Step: 593...  Training loss: 5.7264...  1.5217 sec/batch
Epoch: 10/30...  Training Step: 594...  Training loss: 5.7192...  1.4505 sec/batch
Epoch: 10/30...  Training Step: 595...  Training loss: 5.7580...  1.3161 sec/batch
Epoch: 10/30...  Training Step: 596...  Training loss: 5.7499...  1.3416 sec/batch
Epoch: 10/30...  Training Step: 597...  Training loss: 5.7612...  1.3525 sec/batch
Epoch: 10/30...  Training Step: 598...  Training loss: 5.7617...  1.4168 sec/batch
Epoch: 10/30...  Training Step: 599...  Training loss: 5.7339...  1.4515 sec/batch
Epoch: 10/30...  Training Step: 600...  Training loss: 5.7536...  1.3975 sec/batch
Epoch: 10/30...  Training Step: 601...  Training loss: 5.7210...  1.4265 sec/batch
Epoch: 10/30...  Training Step: 602...  Training loss: 5.7168...  1.4020 sec/batch
Epoch: 10/30...  Training Step: 603...  Training loss: 5.7092...  1.3544 sec/batch
Epoch: 10/30...  Training Step: 604...  Training loss: 5.7245...  1.3449 sec/batch
Epoch: 10/30...  Training Step: 605...  Training loss: 5.7177...  1.4684 sec/batch
Epoch: 10/30...  Training Step: 606...  Training loss: 5.7216...  1.5082 sec/batch
Epoch: 10/30...  Training Step: 607...  Training loss: 5.6860...  1.6173 sec/batch
Epoch: 10/30...  Training Step: 608...  Training loss: 5.7187...  1.4381 sec/batch
Epoch: 10/30...  Training Step: 609...  Training loss: 5.6911...  1.3877 sec/batch
Epoch: 10/30...  Training Step: 610...  Training loss: 5.7233...  1.4890 sec/batch
Epoch: 10/30...  Training Step: 611...  Training loss: 5.7225...  1.5778 sec/batch
Epoch: 10/30...  Training Step: 612...  Training loss: 5.6778...  1.5817 sec/batch
Epoch: 10/30...  Training Step: 613...  Training loss: 5.7147...  1.5480 sec/batch
Epoch: 10/30...  Training Step: 614...  Training loss: 5.6896...  1.4015 sec/batch
Epoch: 10/30...  Training Step: 615...  Training loss: 5.6950...  1.4987 sec/batch
Epoch: 10/30...  Training Step: 616...  Training loss: 5.6904...  1.5565 sec/batch
Epoch: 10/30...  Training Step: 617...  Training loss: 5.6799...  1.4164 sec/batch
Epoch: 10/30...  Training Step: 618...  Training loss: 5.6887...  1.4515 sec/batch
Epoch: 10/30...  Training Step: 619...  Training loss: 5.6718...  1.4546 sec/batch
Epoch: 10/30...  Training Step: 620...  Training loss: 5.6458...  1.3453 sec/batch
Epoch: 11/30...  Training Step: 621...  Training loss: 5.8020...  1.5491 sec/batch
Epoch: 11/30...  Training Step: 622...  Training loss: 5.6460...  1.4413 sec/batch
Epoch: 11/30...  Training Step: 623...  Training loss: 5.6478...  1.4672 sec/batch
Epoch: 11/30...  Training Step: 624...  Training loss: 5.6926...  1.4316 sec/batch
Epoch: 11/30...  Training Step: 625...  Training loss: 5.6578...  1.5119 sec/batch
Epoch: 11/30...  Training Step: 626...  Training loss: 5.6460...  1.3354 sec/batch
Epoch: 11/30...  Training Step: 627...  Training loss: 5.6445...  1.5124 sec/batch
Epoch: 11/30...  Training Step: 628...  Training loss: 5.6597...  1.3516 sec/batch
Epoch: 11/30...  Training Step: 629...  Training loss: 5.6412...  1.2710 sec/batch
Epoch: 11/30...  Training Step: 630...  Training loss: 5.6394...  1.3921 sec/batch
Epoch: 11/30...  Training Step: 631...  Training loss: 5.6222...  1.3949 sec/batch
Epoch: 11/30...  Training Step: 632...  Training loss: 5.6286...  1.3040 sec/batch
Epoch: 11/30...  Training Step: 633...  Training loss: 5.6494...  1.3862 sec/batch
Epoch: 11/30...  Training Step: 634...  Training loss: 5.6330...  1.3997 sec/batch
Epoch: 11/30...  Training Step: 635...  Training loss: 5.6261...  1.3382 sec/batch
Epoch: 11/30...  Training Step: 636...  Training loss: 5.6095...  1.4885 sec/batch
Epoch: 11/30...  Training Step: 637...  Training loss: 5.6189...  1.4366 sec/batch
Epoch: 11/30...  Training Step: 638...  Training loss: 5.6213...  1.3775 sec/batch
Epoch: 11/30...  Training Step: 639...  Training loss: 5.5788...  1.3721 sec/batch
Epoch: 11/30...  Training Step: 640...  Training loss: 5.6014...  1.3611 sec/batch
Epoch: 11/30...  Training Step: 641...  Training loss: 5.5970...  1.4230 sec/batch
Epoch: 11/30...  Training Step: 642...  Training loss: 5.6094...  1.3847 sec/batch
Epoch: 11/30...  Training Step: 643...  Training loss: 5.6312...  1.3804 sec/batch
Epoch: 11/30...  Training Step: 644...  Training loss: 5.6394...  1.3881 sec/batch
Epoch: 11/30...  Training Step: 645...  Training loss: 5.6616...  1.4651 sec/batch
Epoch: 11/30...  Training Step: 646...  Training loss: 5.6500...  1.3582 sec/batch
Epoch: 11/30...  Training Step: 647...  Training loss: 5.6253...  1.4888 sec/batch
Epoch: 11/30...  Training Step: 648...  Training loss: 5.6408...  1.3522 sec/batch
Epoch: 11/30...  Training Step: 649...  Training loss: 5.6119...  1.3939 sec/batch
Epoch: 11/30...  Training Step: 650...  Training loss: 5.6099...  1.3841 sec/batch
Epoch: 11/30...  Training Step: 651...  Training loss: 5.5861...  1.4673 sec/batch
Epoch: 11/30...  Training Step: 652...  Training loss: 5.5673...  1.3925 sec/batch
Epoch: 11/30...  Training Step: 653...  Training loss: 5.5661...  1.3928 sec/batch
Epoch: 11/30...  Training Step: 654...  Training loss: 5.5692...  1.4553 sec/batch
Epoch: 11/30...  Training Step: 655...  Training loss: 5.5590...  1.2961 sec/batch
Epoch: 11/30...  Training Step: 656...  Training loss: 5.5588...  1.2340 sec/batch
Epoch: 11/30...  Training Step: 657...  Training loss: 5.5923...  1.3368 sec/batch
Epoch: 11/30...  Training Step: 658...  Training loss: 5.5860...  1.3540 sec/batch
Epoch: 11/30...  Training Step: 659...  Training loss: 5.5926...  1.4794 sec/batch
Epoch: 11/30...  Training Step: 660...  Training loss: 5.5936...  1.3961 sec/batch
Epoch: 11/30...  Training Step: 661...  Training loss: 5.5683...  1.3781 sec/batch
Epoch: 11/30...  Training Step: 662...  Training loss: 5.5826...  1.3707 sec/batch
Epoch: 11/30...  Training Step: 663...  Training loss: 5.5590...  1.4166 sec/batch
Epoch: 11/30...  Training Step: 664...  Training loss: 5.5454...  1.3323 sec/batch
Epoch: 11/30...  Training Step: 665...  Training loss: 5.5508...  1.3249 sec/batch
Epoch: 11/30...  Training Step: 666...  Training loss: 5.5628...  1.4749 sec/batch
Epoch: 11/30...  Training Step: 667...  Training loss: 5.5423...  1.4192 sec/batch
Epoch: 11/30...  Training Step: 668...  Training loss: 5.5696...  1.3382 sec/batch
Epoch: 11/30...  Training Step: 669...  Training loss: 5.5189...  1.2506 sec/batch
Epoch: 11/30...  Training Step: 670...  Training loss: 5.5509...  1.3649 sec/batch
Epoch: 11/30...  Training Step: 671...  Training loss: 5.5449...  1.3549 sec/batch
Epoch: 11/30...  Training Step: 672...  Training loss: 5.5692...  1.4931 sec/batch
Epoch: 11/30...  Training Step: 673...  Training loss: 5.5718...  1.4690 sec/batch
Epoch: 11/30...  Training Step: 674...  Training loss: 5.5226...  1.3608 sec/batch
Epoch: 11/30...  Training Step: 675...  Training loss: 5.5528...  1.3775 sec/batch
Epoch: 11/30...  Training Step: 676...  Training loss: 5.5432...  1.4001 sec/batch
Epoch: 11/30...  Training Step: 677...  Training loss: 5.5330...  1.3888 sec/batch
Epoch: 11/30...  Training Step: 678...  Training loss: 5.5407...  1.3919 sec/batch
Epoch: 11/30...  Training Step: 679...  Training loss: 5.5241...  1.3804 sec/batch
Epoch: 11/30...  Training Step: 680...  Training loss: 5.5292...  1.4011 sec/batch
Epoch: 11/30...  Training Step: 681...  Training loss: 5.5201...  1.3877 sec/batch
Epoch: 11/30...  Training Step: 682...  Training loss: 5.4845...  1.3914 sec/batch
Epoch: 12/30...  Training Step: 683...  Training loss: 5.6404...  1.6275 sec/batch
Epoch: 12/30...  Training Step: 684...  Training loss: 5.4866...  1.4377 sec/batch
Epoch: 12/30...  Training Step: 685...  Training loss: 5.4952...  1.2916 sec/batch
Epoch: 12/30...  Training Step: 686...  Training loss: 5.5391...  1.4109 sec/batch
Epoch: 12/30...  Training Step: 687...  Training loss: 5.5027...  1.3583 sec/batch
Epoch: 12/30...  Training Step: 688...  Training loss: 5.5081...  1.5278 sec/batch
Epoch: 12/30...  Training Step: 689...  Training loss: 5.4885...  1.3421 sec/batch
Epoch: 12/30...  Training Step: 690...  Training loss: 5.5038...  1.5251 sec/batch
Epoch: 12/30...  Training Step: 691...  Training loss: 5.4931...  1.4325 sec/batch
Epoch: 12/30...  Training Step: 692...  Training loss: 5.4874...  1.5755 sec/batch
Epoch: 12/30...  Training Step: 693...  Training loss: 5.4686...  1.3011 sec/batch
Epoch: 12/30...  Training Step: 694...  Training loss: 5.4690...  1.4688 sec/batch
Epoch: 12/30...  Training Step: 695...  Training loss: 5.5012...  1.4078 sec/batch
Epoch: 12/30...  Training Step: 696...  Training loss: 5.4782...  1.4014 sec/batch
Epoch: 12/30...  Training Step: 697...  Training loss: 5.4833...  1.3340 sec/batch
Epoch: 12/30...  Training Step: 698...  Training loss: 5.4574...  1.4262 sec/batch
Epoch: 12/30...  Training Step: 699...  Training loss: 5.4827...  1.3100 sec/batch
Epoch: 12/30...  Training Step: 700...  Training loss: 5.4733...  1.2806 sec/batch
Epoch: 12/30...  Training Step: 701...  Training loss: 5.4219...  1.3139 sec/batch
Epoch: 12/30...  Training Step: 702...  Training loss: 5.4564...  1.4096 sec/batch
Epoch: 12/30...  Training Step: 703...  Training loss: 5.4357...  1.4907 sec/batch
Epoch: 12/30...  Training Step: 704...  Training loss: 5.4714...  1.3426 sec/batch
Epoch: 12/30...  Training Step: 705...  Training loss: 5.4696...  1.4333 sec/batch
Epoch: 12/30...  Training Step: 706...  Training loss: 5.5061...  1.4052 sec/batch
Epoch: 12/30...  Training Step: 707...  Training loss: 5.5000...  1.2945 sec/batch
Epoch: 12/30...  Training Step: 708...  Training loss: 5.5071...  1.4434 sec/batch
Epoch: 12/30...  Training Step: 709...  Training loss: 5.4644...  1.4174 sec/batch
Epoch: 12/30...  Training Step: 710...  Training loss: 5.4853...  1.5289 sec/batch
Epoch: 12/30...  Training Step: 711...  Training loss: 5.4681...  1.5190 sec/batch
Epoch: 12/30...  Training Step: 712...  Training loss: 5.4518...  1.5632 sec/batch
Epoch: 12/30...  Training Step: 713...  Training loss: 5.4404...  1.6450 sec/batch
Epoch: 12/30...  Training Step: 714...  Training loss: 5.4171...  1.4386 sec/batch
Epoch: 12/30...  Training Step: 715...  Training loss: 5.4130...  1.3989 sec/batch
Epoch: 12/30...  Training Step: 716...  Training loss: 5.4203...  1.6801 sec/batch
Epoch: 12/30...  Training Step: 717...  Training loss: 5.3913...  1.3985 sec/batch
Epoch: 12/30...  Training Step: 718...  Training loss: 5.4032...  1.3319 sec/batch
Epoch: 12/30...  Training Step: 719...  Training loss: 5.4407...  1.4446 sec/batch
Epoch: 12/30...  Training Step: 720...  Training loss: 5.4287...  1.6326 sec/batch
Epoch: 12/30...  Training Step: 721...  Training loss: 5.4438...  1.7452 sec/batch
Epoch: 12/30...  Training Step: 722...  Training loss: 5.4530...  1.7382 sec/batch
Epoch: 12/30...  Training Step: 723...  Training loss: 5.4120...  1.5780 sec/batch
Epoch: 12/30...  Training Step: 724...  Training loss: 5.4271...  1.3456 sec/batch
Epoch: 12/30...  Training Step: 725...  Training loss: 5.4107...  1.4217 sec/batch
Epoch: 12/30...  Training Step: 726...  Training loss: 5.3878...  1.3841 sec/batch
Epoch: 12/30...  Training Step: 727...  Training loss: 5.3947...  1.4758 sec/batch
Epoch: 12/30...  Training Step: 728...  Training loss: 5.4016...  1.5151 sec/batch
Epoch: 12/30...  Training Step: 729...  Training loss: 5.3893...  1.5133 sec/batch
Epoch: 12/30...  Training Step: 730...  Training loss: 5.4218...  1.3134 sec/batch
Epoch: 12/30...  Training Step: 731...  Training loss: 5.3528...  1.6004 sec/batch
Epoch: 12/30...  Training Step: 732...  Training loss: 5.3913...  1.5946 sec/batch
Epoch: 12/30...  Training Step: 733...  Training loss: 5.3794...  1.4488 sec/batch
Epoch: 12/30...  Training Step: 734...  Training loss: 5.3973...  1.4675 sec/batch
Epoch: 12/30...  Training Step: 735...  Training loss: 5.4200...  1.3069 sec/batch
Epoch: 12/30...  Training Step: 736...  Training loss: 5.3653...  1.4537 sec/batch
Epoch: 12/30...  Training Step: 737...  Training loss: 5.3899...  1.3527 sec/batch
Epoch: 12/30...  Training Step: 738...  Training loss: 5.3816...  1.3245 sec/batch
Epoch: 12/30...  Training Step: 739...  Training loss: 5.3817...  1.3718 sec/batch
Epoch: 12/30...  Training Step: 740...  Training loss: 5.3839...  1.3536 sec/batch
Epoch: 12/30...  Training Step: 741...  Training loss: 5.3682...  1.3280 sec/batch
Epoch: 12/30...  Training Step: 742...  Training loss: 5.3866...  1.4345 sec/batch
Epoch: 12/30...  Training Step: 743...  Training loss: 5.3663...  1.5936 sec/batch
Epoch: 12/30...  Training Step: 744...  Training loss: 5.3245...  1.3671 sec/batch
Epoch: 13/30...  Training Step: 745...  Training loss: 5.4892...  1.4065 sec/batch
Epoch: 13/30...  Training Step: 746...  Training loss: 5.3330...  1.3156 sec/batch
Epoch: 13/30...  Training Step: 747...  Training loss: 5.3390...  1.3885 sec/batch
Epoch: 13/30...  Training Step: 748...  Training loss: 5.3844...  1.3718 sec/batch
Epoch: 13/30...  Training Step: 749...  Training loss: 5.3437...  1.4078 sec/batch
Epoch: 13/30...  Training Step: 750...  Training loss: 5.3401...  1.4839 sec/batch
Epoch: 13/30...  Training Step: 751...  Training loss: 5.3348...  1.4171 sec/batch
Epoch: 13/30...  Training Step: 752...  Training loss: 5.3396...  1.3471 sec/batch
Epoch: 13/30...  Training Step: 753...  Training loss: 5.3247...  1.4176 sec/batch
Epoch: 13/30...  Training Step: 754...  Training loss: 5.3385...  1.3339 sec/batch
Epoch: 13/30...  Training Step: 755...  Training loss: 5.3060...  1.2689 sec/batch
Epoch: 13/30...  Training Step: 756...  Training loss: 5.3026...  1.3174 sec/batch
Epoch: 13/30...  Training Step: 757...  Training loss: 5.3447...  1.4559 sec/batch
Epoch: 13/30...  Training Step: 758...  Training loss: 5.3238...  1.5531 sec/batch
Epoch: 13/30...  Training Step: 759...  Training loss: 5.3189...  1.3429 sec/batch
Epoch: 13/30...  Training Step: 760...  Training loss: 5.2993...  1.4635 sec/batch
Epoch: 13/30...  Training Step: 761...  Training loss: 5.3247...  1.4816 sec/batch
Epoch: 13/30...  Training Step: 762...  Training loss: 5.3065...  1.5952 sec/batch
Epoch: 13/30...  Training Step: 763...  Training loss: 5.2695...  1.4217 sec/batch
Epoch: 13/30...  Training Step: 764...  Training loss: 5.2877...  1.3201 sec/batch
Epoch: 13/30...  Training Step: 765...  Training loss: 5.2686...  1.4646 sec/batch
Epoch: 13/30...  Training Step: 766...  Training loss: 5.2959...  1.4125 sec/batch
Epoch: 13/30...  Training Step: 767...  Training loss: 5.3084...  1.3088 sec/batch
Epoch: 13/30...  Training Step: 768...  Training loss: 5.3402...  1.3683 sec/batch
Epoch: 13/30...  Training Step: 769...  Training loss: 5.3449...  1.4524 sec/batch
Epoch: 13/30...  Training Step: 770...  Training loss: 5.3436...  1.5329 sec/batch
Epoch: 13/30...  Training Step: 771...  Training loss: 5.2953...  1.3256 sec/batch
Epoch: 13/30...  Training Step: 772...  Training loss: 5.3160...  1.4450 sec/batch
Epoch: 13/30...  Training Step: 773...  Training loss: 5.2883...  1.4941 sec/batch
Epoch: 13/30...  Training Step: 774...  Training loss: 5.2853...  1.5891 sec/batch
Epoch: 13/30...  Training Step: 775...  Training loss: 5.2587...  1.3521 sec/batch
Epoch: 13/30...  Training Step: 776...  Training loss: 5.2539...  1.4687 sec/batch
Epoch: 13/30...  Training Step: 777...  Training loss: 5.2444...  1.3932 sec/batch
Epoch: 13/30...  Training Step: 778...  Training loss: 5.2512...  1.4905 sec/batch
Epoch: 13/30...  Training Step: 779...  Training loss: 5.2262...  1.3793 sec/batch
Epoch: 13/30...  Training Step: 780...  Training loss: 5.2373...  1.5083 sec/batch
Epoch: 13/30...  Training Step: 781...  Training loss: 5.2859...  1.3907 sec/batch
Epoch: 13/30...  Training Step: 782...  Training loss: 5.2735...  1.3534 sec/batch
Epoch: 13/30...  Training Step: 783...  Training loss: 5.2798...  1.3400 sec/batch
Epoch: 13/30...  Training Step: 784...  Training loss: 5.2915...  1.3616 sec/batch
Epoch: 13/30...  Training Step: 785...  Training loss: 5.2581...  1.5783 sec/batch
Epoch: 13/30...  Training Step: 786...  Training loss: 5.2632...  1.3799 sec/batch
Epoch: 13/30...  Training Step: 787...  Training loss: 5.2562...  1.3766 sec/batch
Epoch: 13/30...  Training Step: 788...  Training loss: 5.2324...  1.4224 sec/batch
Epoch: 13/30...  Training Step: 789...  Training loss: 5.2342...  1.3997 sec/batch
Epoch: 13/30...  Training Step: 790...  Training loss: 5.2580...  1.4845 sec/batch
Epoch: 13/30...  Training Step: 791...  Training loss: 5.2265...  1.5280 sec/batch
Epoch: 13/30...  Training Step: 792...  Training loss: 5.2502...  1.4376 sec/batch
Epoch: 13/30...  Training Step: 793...  Training loss: 5.1833...  1.3776 sec/batch
Epoch: 13/30...  Training Step: 794...  Training loss: 5.2347...  1.3970 sec/batch
Epoch: 13/30...  Training Step: 795...  Training loss: 5.2145...  1.4472 sec/batch
Epoch: 13/30...  Training Step: 796...  Training loss: 5.2340...  1.3706 sec/batch
Epoch: 13/30...  Training Step: 797...  Training loss: 5.2441...  1.3711 sec/batch
Epoch: 13/30...  Training Step: 798...  Training loss: 5.1996...  1.2888 sec/batch
Epoch: 13/30...  Training Step: 799...  Training loss: 5.2295...  1.4351 sec/batch
Epoch: 13/30...  Training Step: 800...  Training loss: 5.2250...  1.3286 sec/batch
Epoch: 13/30...  Training Step: 801...  Training loss: 5.2132...  1.3398 sec/batch
Epoch: 13/30...  Training Step: 802...  Training loss: 5.2117...  1.3009 sec/batch
Epoch: 13/30...  Training Step: 803...  Training loss: 5.2041...  1.2413 sec/batch
Epoch: 13/30...  Training Step: 804...  Training loss: 5.2142...  1.2674 sec/batch
Epoch: 13/30...  Training Step: 805...  Training loss: 5.1938...  1.3519 sec/batch
Epoch: 13/30...  Training Step: 806...  Training loss: 5.1550...  1.4695 sec/batch
Epoch: 14/30...  Training Step: 807...  Training loss: 5.3057...  1.3180 sec/batch
Epoch: 14/30...  Training Step: 808...  Training loss: 5.1575...  1.4060 sec/batch
Epoch: 14/30...  Training Step: 809...  Training loss: 5.1740...  1.4169 sec/batch
Epoch: 14/30...  Training Step: 810...  Training loss: 5.2217...  1.3716 sec/batch
Epoch: 14/30...  Training Step: 811...  Training loss: 5.1797...  1.4072 sec/batch
Epoch: 14/30...  Training Step: 812...  Training loss: 5.1765...  1.3420 sec/batch
Epoch: 14/30...  Training Step: 813...  Training loss: 5.1626...  1.4547 sec/batch
Epoch: 14/30...  Training Step: 814...  Training loss: 5.1753...  1.3272 sec/batch
Epoch: 14/30...  Training Step: 815...  Training loss: 5.1707...  1.2574 sec/batch
Epoch: 14/30...  Training Step: 816...  Training loss: 5.1702...  1.4660 sec/batch
Epoch: 14/30...  Training Step: 817...  Training loss: 5.1407...  1.4640 sec/batch
Epoch: 14/30...  Training Step: 818...  Training loss: 5.1447...  1.4953 sec/batch
Epoch: 14/30...  Training Step: 819...  Training loss: 5.1695...  1.3301 sec/batch
Epoch: 14/30...  Training Step: 820...  Training loss: 5.1500...  1.2784 sec/batch
Epoch: 14/30...  Training Step: 821...  Training loss: 5.1508...  1.3269 sec/batch
Epoch: 14/30...  Training Step: 822...  Training loss: 5.1367...  1.4004 sec/batch
Epoch: 14/30...  Training Step: 823...  Training loss: 5.1568...  1.5412 sec/batch
Epoch: 14/30...  Training Step: 824...  Training loss: 5.1473...  1.4744 sec/batch
Epoch: 14/30...  Training Step: 825...  Training loss: 5.1024...  1.3299 sec/batch
Epoch: 14/30...  Training Step: 826...  Training loss: 5.1299...  1.4171 sec/batch
Epoch: 14/30...  Training Step: 827...  Training loss: 5.0985...  1.4795 sec/batch
Epoch: 14/30...  Training Step: 828...  Training loss: 5.1416...  1.4669 sec/batch
Epoch: 14/30...  Training Step: 829...  Training loss: 5.1456...  1.3565 sec/batch
Epoch: 14/30...  Training Step: 830...  Training loss: 5.1787...  1.3388 sec/batch
Epoch: 14/30...  Training Step: 831...  Training loss: 5.1806...  1.4143 sec/batch
Epoch: 14/30...  Training Step: 832...  Training loss: 5.1804...  1.3416 sec/batch
Epoch: 14/30...  Training Step: 833...  Training loss: 5.1502...  1.4963 sec/batch
Epoch: 14/30...  Training Step: 834...  Training loss: 5.1604...  1.2426 sec/batch
Epoch: 14/30...  Training Step: 835...  Training loss: 5.1362...  1.4211 sec/batch
Epoch: 14/30...  Training Step: 836...  Training loss: 5.1258...  1.5124 sec/batch
Epoch: 14/30...  Training Step: 837...  Training loss: 5.0889...  1.2907 sec/batch
Epoch: 14/30...  Training Step: 838...  Training loss: 5.0841...  1.3653 sec/batch
Epoch: 14/30...  Training Step: 839...  Training loss: 5.0861...  1.3453 sec/batch
Epoch: 14/30...  Training Step: 840...  Training loss: 5.0838...  1.3233 sec/batch
Epoch: 14/30...  Training Step: 841...  Training loss: 5.0716...  1.3711 sec/batch
Epoch: 14/30...  Training Step: 842...  Training loss: 5.0852...  1.3825 sec/batch
Epoch: 14/30...  Training Step: 843...  Training loss: 5.1139...  1.3470 sec/batch
Epoch: 14/30...  Training Step: 844...  Training loss: 5.1193...  1.4319 sec/batch
Epoch: 14/30...  Training Step: 845...  Training loss: 5.1060...  1.4086 sec/batch
Epoch: 14/30...  Training Step: 846...  Training loss: 5.1395...  1.5174 sec/batch
Epoch: 14/30...  Training Step: 847...  Training loss: 5.0905...  1.3373 sec/batch
Epoch: 14/30...  Training Step: 848...  Training loss: 5.0976...  1.4250 sec/batch
Epoch: 14/30...  Training Step: 849...  Training loss: 5.0925...  1.5479 sec/batch
Epoch: 14/30...  Training Step: 850...  Training loss: 5.0725...  1.3933 sec/batch
Epoch: 14/30...  Training Step: 851...  Training loss: 5.0640...  1.4238 sec/batch
Epoch: 14/30...  Training Step: 852...  Training loss: 5.0880...  1.3847 sec/batch
Epoch: 14/30...  Training Step: 853...  Training loss: 5.0554...  1.3645 sec/batch
Epoch: 14/30...  Training Step: 854...  Training loss: 5.0911...  1.4195 sec/batch
Epoch: 14/30...  Training Step: 855...  Training loss: 5.0274...  1.4820 sec/batch
Epoch: 14/30...  Training Step: 856...  Training loss: 5.0735...  1.3606 sec/batch
Epoch: 14/30...  Training Step: 857...  Training loss: 5.0436...  1.4269 sec/batch
Epoch: 14/30...  Training Step: 858...  Training loss: 5.0728...  1.5071 sec/batch
Epoch: 14/30...  Training Step: 859...  Training loss: 5.0854...  1.5021 sec/batch
Epoch: 14/30...  Training Step: 860...  Training loss: 5.0346...  1.3091 sec/batch
Epoch: 14/30...  Training Step: 861...  Training loss: 5.0675...  1.3073 sec/batch
Epoch: 14/30...  Training Step: 862...  Training loss: 5.0689...  1.3544 sec/batch
Epoch: 14/30...  Training Step: 863...  Training loss: 5.0501...  1.3587 sec/batch
Epoch: 14/30...  Training Step: 864...  Training loss: 5.0532...  1.4167 sec/batch
Epoch: 14/30...  Training Step: 865...  Training loss: 5.0387...  1.3313 sec/batch
Epoch: 14/30...  Training Step: 866...  Training loss: 5.0554...  1.4221 sec/batch
Epoch: 14/30...  Training Step: 867...  Training loss: 5.0316...  1.5384 sec/batch
Epoch: 14/30...  Training Step: 868...  Training loss: 4.9966...  1.3933 sec/batch
Epoch: 15/30...  Training Step: 869...  Training loss: 5.1650...  1.3448 sec/batch
Epoch: 15/30...  Training Step: 870...  Training loss: 4.9999...  1.5121 sec/batch
Epoch: 15/30...  Training Step: 871...  Training loss: 5.0254...  1.2921 sec/batch
Epoch: 15/30...  Training Step: 872...  Training loss: 5.0689...  1.2864 sec/batch
Epoch: 15/30...  Training Step: 873...  Training loss: 5.0346...  1.4815 sec/batch
Epoch: 15/30...  Training Step: 874...  Training loss: 5.0181...  1.3971 sec/batch
Epoch: 15/30...  Training Step: 875...  Training loss: 5.0041...  1.3965 sec/batch
Epoch: 15/30...  Training Step: 876...  Training loss: 5.0347...  1.4514 sec/batch
Epoch: 15/30...  Training Step: 877...  Training loss: 5.0123...  1.4349 sec/batch
Epoch: 15/30...  Training Step: 878...  Training loss: 5.0162...  1.5495 sec/batch
Epoch: 15/30...  Training Step: 879...  Training loss: 4.9926...  1.2801 sec/batch
Epoch: 15/30...  Training Step: 880...  Training loss: 4.9994...  1.3364 sec/batch
Epoch: 15/30...  Training Step: 881...  Training loss: 5.0285...  1.5006 sec/batch
Epoch: 15/30...  Training Step: 882...  Training loss: 5.0067...  1.4495 sec/batch
Epoch: 15/30...  Training Step: 883...  Training loss: 5.0035...  1.2492 sec/batch
Epoch: 15/30...  Training Step: 884...  Training loss: 4.9845...  1.4096 sec/batch
Epoch: 15/30...  Training Step: 885...  Training loss: 4.9978...  1.3879 sec/batch
Epoch: 15/30...  Training Step: 886...  Training loss: 4.9941...  1.3360 sec/batch
Epoch: 15/30...  Training Step: 887...  Training loss: 4.9467...  1.3894 sec/batch
Epoch: 15/30...  Training Step: 888...  Training loss: 4.9752...  1.3838 sec/batch
Epoch: 15/30...  Training Step: 889...  Training loss: 4.9547...  1.3965 sec/batch
Epoch: 15/30...  Training Step: 890...  Training loss: 4.9917...  1.3008 sec/batch
Epoch: 15/30...  Training Step: 891...  Training loss: 5.0020...  1.2802 sec/batch
Epoch: 15/30...  Training Step: 892...  Training loss: 5.0291...  1.3452 sec/batch
Epoch: 15/30...  Training Step: 893...  Training loss: 5.0293...  1.2630 sec/batch
Epoch: 15/30...  Training Step: 894...  Training loss: 5.0312...  1.5180 sec/batch
Epoch: 15/30...  Training Step: 895...  Training loss: 4.9935...  1.4801 sec/batch
Epoch: 15/30...  Training Step: 896...  Training loss: 5.0071...  1.4249 sec/batch
Epoch: 15/30...  Training Step: 897...  Training loss: 4.9729...  1.3315 sec/batch
Epoch: 15/30...  Training Step: 898...  Training loss: 4.9746...  1.4873 sec/batch
Epoch: 15/30...  Training Step: 899...  Training loss: 4.9389...  1.3192 sec/batch
Epoch: 15/30...  Training Step: 900...  Training loss: 4.9376...  1.3792 sec/batch
Epoch: 15/30...  Training Step: 901...  Training loss: 4.9261...  1.4341 sec/batch
Epoch: 15/30...  Training Step: 902...  Training loss: 4.9279...  1.5157 sec/batch
Epoch: 15/30...  Training Step: 903...  Training loss: 4.9157...  1.3581 sec/batch
Epoch: 15/30...  Training Step: 904...  Training loss: 4.9196...  1.5133 sec/batch
Epoch: 15/30...  Training Step: 905...  Training loss: 4.9613...  1.3427 sec/batch
Epoch: 15/30...  Training Step: 906...  Training loss: 4.9491...  1.4122 sec/batch
Epoch: 15/30...  Training Step: 907...  Training loss: 4.9615...  1.4308 sec/batch
Epoch: 15/30...  Training Step: 908...  Training loss: 4.9897...  1.5670 sec/batch
Epoch: 15/30...  Training Step: 909...  Training loss: 4.9341...  1.3297 sec/batch
Epoch: 15/30...  Training Step: 910...  Training loss: 4.9581...  1.3941 sec/batch
Epoch: 15/30...  Training Step: 911...  Training loss: 4.9469...  1.3947 sec/batch
Epoch: 15/30...  Training Step: 912...  Training loss: 4.9231...  1.3812 sec/batch
Epoch: 15/30...  Training Step: 913...  Training loss: 4.9305...  1.5794 sec/batch
Epoch: 15/30...  Training Step: 914...  Training loss: 4.9536...  1.4398 sec/batch
Epoch: 15/30...  Training Step: 915...  Training loss: 4.9214...  1.3379 sec/batch
Epoch: 15/30...  Training Step: 916...  Training loss: 4.9536...  1.3696 sec/batch
Epoch: 15/30...  Training Step: 917...  Training loss: 4.8763...  1.4233 sec/batch
Epoch: 15/30...  Training Step: 918...  Training loss: 4.9344...  1.2941 sec/batch
Epoch: 15/30...  Training Step: 919...  Training loss: 4.9085...  1.3283 sec/batch
Epoch: 15/30...  Training Step: 920...  Training loss: 4.9394...  1.2805 sec/batch
Epoch: 15/30...  Training Step: 921...  Training loss: 4.9327...  1.5064 sec/batch
Epoch: 15/30...  Training Step: 922...  Training loss: 4.9072...  1.4153 sec/batch
Epoch: 15/30...  Training Step: 923...  Training loss: 4.9347...  1.3609 sec/batch
Epoch: 15/30...  Training Step: 924...  Training loss: 4.9311...  1.3401 sec/batch
Epoch: 15/30...  Training Step: 925...  Training loss: 4.9243...  1.3258 sec/batch
Epoch: 15/30...  Training Step: 926...  Training loss: 4.9224...  1.2956 sec/batch
Epoch: 15/30...  Training Step: 927...  Training loss: 4.9093...  1.4605 sec/batch
Epoch: 15/30...  Training Step: 928...  Training loss: 4.9159...  1.4865 sec/batch
Epoch: 15/30...  Training Step: 929...  Training loss: 4.9027...  1.4403 sec/batch
Epoch: 15/30...  Training Step: 930...  Training loss: 4.8740...  1.4303 sec/batch
Epoch: 16/30...  Training Step: 931...  Training loss: 5.0090...  1.2784 sec/batch
Epoch: 16/30...  Training Step: 932...  Training loss: 4.8699...  1.5347 sec/batch
Epoch: 16/30...  Training Step: 933...  Training loss: 4.8913...  1.3910 sec/batch
Epoch: 16/30...  Training Step: 934...  Training loss: 4.9374...  1.3468 sec/batch
Epoch: 16/30...  Training Step: 935...  Training loss: 4.8937...  1.4021 sec/batch
Epoch: 16/30...  Training Step: 936...  Training loss: 4.8825...  1.5291 sec/batch
Epoch: 16/30...  Training Step: 937...  Training loss: 4.8671...  1.4813 sec/batch
Epoch: 16/30...  Training Step: 938...  Training loss: 4.8906...  1.3209 sec/batch
Epoch: 16/30...  Training Step: 939...  Training loss: 4.8776...  1.5409 sec/batch
Epoch: 16/30...  Training Step: 940...  Training loss: 4.8889...  1.3168 sec/batch
Epoch: 16/30...  Training Step: 941...  Training loss: 4.8557...  1.3236 sec/batch
Epoch: 16/30...  Training Step: 942...  Training loss: 4.8647...  1.3365 sec/batch
Epoch: 16/30...  Training Step: 943...  Training loss: 4.8967...  1.5332 sec/batch
Epoch: 16/30...  Training Step: 944...  Training loss: 4.8627...  1.3261 sec/batch
Epoch: 16/30...  Training Step: 945...  Training loss: 4.8610...  1.3667 sec/batch
Epoch: 16/30...  Training Step: 946...  Training loss: 4.8500...  1.3977 sec/batch
Epoch: 16/30...  Training Step: 947...  Training loss: 4.8681...  1.2931 sec/batch
Epoch: 16/30...  Training Step: 948...  Training loss: 4.8543...  1.5280 sec/batch
Epoch: 16/30...  Training Step: 949...  Training loss: 4.8044...  1.3179 sec/batch
Epoch: 16/30...  Training Step: 950...  Training loss: 4.8546...  1.4717 sec/batch
Epoch: 16/30...  Training Step: 951...  Training loss: 4.8197...  1.3254 sec/batch
Epoch: 16/30...  Training Step: 952...  Training loss: 4.8602...  1.3720 sec/batch
Epoch: 16/30...  Training Step: 953...  Training loss: 4.8644...  1.3565 sec/batch
Epoch: 16/30...  Training Step: 954...  Training loss: 4.8979...  1.3488 sec/batch
Epoch: 16/30...  Training Step: 955...  Training loss: 4.9072...  1.3343 sec/batch
Epoch: 16/30...  Training Step: 956...  Training loss: 4.9083...  1.4381 sec/batch
Epoch: 16/30...  Training Step: 957...  Training loss: 4.8639...  1.5689 sec/batch
Epoch: 16/30...  Training Step: 958...  Training loss: 4.8876...  1.5074 sec/batch
Epoch: 16/30...  Training Step: 959...  Training loss: 4.8500...  1.5308 sec/batch
Epoch: 16/30...  Training Step: 960...  Training loss: 4.8440...  1.5491 sec/batch
Epoch: 16/30...  Training Step: 961...  Training loss: 4.8218...  1.3590 sec/batch
Epoch: 16/30...  Training Step: 962...  Training loss: 4.8110...  1.2825 sec/batch
Epoch: 16/30...  Training Step: 963...  Training loss: 4.7987...  1.2672 sec/batch
Epoch: 16/30...  Training Step: 964...  Training loss: 4.8229...  1.4218 sec/batch
Epoch: 16/30...  Training Step: 965...  Training loss: 4.7860...  1.5335 sec/batch
Epoch: 16/30...  Training Step: 966...  Training loss: 4.8104...  1.4154 sec/batch
Epoch: 16/30...  Training Step: 967...  Training loss: 4.8429...  1.5967 sec/batch
Epoch: 16/30...  Training Step: 968...  Training loss: 4.8362...  1.5013 sec/batch
Epoch: 16/30...  Training Step: 969...  Training loss: 4.8470...  1.2851 sec/batch
Epoch: 16/30...  Training Step: 970...  Training loss: 4.8688...  1.3513 sec/batch
Epoch: 16/30...  Training Step: 971...  Training loss: 4.8106...  1.3894 sec/batch
Epoch: 16/30...  Training Step: 972...  Training loss: 4.8356...  1.3756 sec/batch
Epoch: 16/30...  Training Step: 973...  Training loss: 4.8154...  1.3296 sec/batch
Epoch: 16/30...  Training Step: 974...  Training loss: 4.8040...  1.3709 sec/batch
Epoch: 16/30...  Training Step: 975...  Training loss: 4.8017...  1.4204 sec/batch
Epoch: 16/30...  Training Step: 976...  Training loss: 4.8187...  1.3255 sec/batch
Epoch: 16/30...  Training Step: 977...  Training loss: 4.7917...  1.4516 sec/batch
Epoch: 16/30...  Training Step: 978...  Training loss: 4.8270...  1.4487 sec/batch
Epoch: 16/30...  Training Step: 979...  Training loss: 4.7524...  1.4332 sec/batch
Epoch: 16/30...  Training Step: 980...  Training loss: 4.8094...  1.4223 sec/batch
Epoch: 16/30...  Training Step: 981...  Training loss: 4.7807...  1.4573 sec/batch
Epoch: 16/30...  Training Step: 982...  Training loss: 4.8195...  1.4534 sec/batch
Epoch: 16/30...  Training Step: 983...  Training loss: 4.8057...  1.4561 sec/batch
Epoch: 16/30...  Training Step: 984...  Training loss: 4.7909...  1.3518 sec/batch
Epoch: 16/30...  Training Step: 985...  Training loss: 4.8031...  1.3183 sec/batch
Epoch: 16/30...  Training Step: 986...  Training loss: 4.8144...  1.5057 sec/batch
Epoch: 16/30...  Training Step: 987...  Training loss: 4.8010...  1.4887 sec/batch
Epoch: 16/30...  Training Step: 988...  Training loss: 4.7907...  1.3912 sec/batch
Epoch: 16/30...  Training Step: 989...  Training loss: 4.7823...  1.4129 sec/batch
Epoch: 16/30...  Training Step: 990...  Training loss: 4.7989...  1.2938 sec/batch
Epoch: 16/30...  Training Step: 991...  Training loss: 4.7739...  1.3321 sec/batch
Epoch: 16/30...  Training Step: 992...  Training loss: 4.7395...  1.4391 sec/batch
Epoch: 17/30...  Training Step: 993...  Training loss: 4.8770...  1.3217 sec/batch
Epoch: 17/30...  Training Step: 994...  Training loss: 4.7507...  1.2902 sec/batch
Epoch: 17/30...  Training Step: 995...  Training loss: 4.7595...  1.5239 sec/batch
Epoch: 17/30...  Training Step: 996...  Training loss: 4.8143...  1.3864 sec/batch
Epoch: 17/30...  Training Step: 997...  Training loss: 4.7713...  1.3448 sec/batch
Epoch: 17/30...  Training Step: 998...  Training loss: 4.7611...  1.3783 sec/batch
Epoch: 17/30...  Training Step: 999...  Training loss: 4.7451...  1.3542 sec/batch
Epoch: 17/30...  Training Step: 1000...  Training loss: 4.7765...  1.3338 sec/batch
Epoch: 17/30...  Training Step: 1001...  Training loss: 4.7574...  1.4473 sec/batch
Epoch: 17/30...  Training Step: 1002...  Training loss: 4.7603...  1.3945 sec/batch
Epoch: 17/30...  Training Step: 1003...  Training loss: 4.7225...  1.3768 sec/batch
Epoch: 17/30...  Training Step: 1004...  Training loss: 4.7410...  1.5896 sec/batch
Epoch: 17/30...  Training Step: 1005...  Training loss: 4.7762...  1.4871 sec/batch
Epoch: 17/30...  Training Step: 1006...  Training loss: 4.7501...  1.5403 sec/batch
Epoch: 17/30...  Training Step: 1007...  Training loss: 4.7427...  1.5240 sec/batch
Epoch: 17/30...  Training Step: 1008...  Training loss: 4.7197...  1.3885 sec/batch
Epoch: 17/30...  Training Step: 1009...  Training loss: 4.7522...  1.4069 sec/batch
Epoch: 17/30...  Training Step: 1010...  Training loss: 4.7269...  1.3902 sec/batch
Epoch: 17/30...  Training Step: 1011...  Training loss: 4.6778...  1.6515 sec/batch
Epoch: 17/30...  Training Step: 1012...  Training loss: 4.7284...  1.3584 sec/batch
Epoch: 17/30...  Training Step: 1013...  Training loss: 4.7070...  1.2632 sec/batch
Epoch: 17/30...  Training Step: 1014...  Training loss: 4.7482...  1.4295 sec/batch
Epoch: 17/30...  Training Step: 1015...  Training loss: 4.7475...  1.2772 sec/batch
Epoch: 17/30...  Training Step: 1016...  Training loss: 4.7740...  1.3711 sec/batch
Epoch: 17/30...  Training Step: 1017...  Training loss: 4.7795...  1.3609 sec/batch
Epoch: 17/30...  Training Step: 1018...  Training loss: 4.7997...  1.3409 sec/batch
Epoch: 17/30...  Training Step: 1019...  Training loss: 4.7464...  1.3537 sec/batch
Epoch: 17/30...  Training Step: 1020...  Training loss: 4.7675...  1.4160 sec/batch
Epoch: 17/30...  Training Step: 1021...  Training loss: 4.7459...  1.3969 sec/batch
Epoch: 17/30...  Training Step: 1022...  Training loss: 4.7360...  1.4375 sec/batch
Epoch: 17/30...  Training Step: 1023...  Training loss: 4.7134...  1.4849 sec/batch
Epoch: 17/30...  Training Step: 1024...  Training loss: 4.6996...  1.3391 sec/batch
Epoch: 17/30...  Training Step: 1025...  Training loss: 4.6932...  1.4634 sec/batch
Epoch: 17/30...  Training Step: 1026...  Training loss: 4.6968...  1.4828 sec/batch
Epoch: 17/30...  Training Step: 1027...  Training loss: 4.6728...  1.4126 sec/batch
Epoch: 17/30...  Training Step: 1028...  Training loss: 4.7025...  1.4769 sec/batch
Epoch: 17/30...  Training Step: 1029...  Training loss: 4.7263...  1.4909 sec/batch
Epoch: 17/30...  Training Step: 1030...  Training loss: 4.7314...  1.4736 sec/batch
Epoch: 17/30...  Training Step: 1031...  Training loss: 4.7297...  1.6549 sec/batch
Epoch: 17/30...  Training Step: 1032...  Training loss: 4.7451...  1.4530 sec/batch
Epoch: 17/30...  Training Step: 1033...  Training loss: 4.7037...  1.4675 sec/batch
Epoch: 17/30...  Training Step: 1034...  Training loss: 4.7245...  1.4314 sec/batch
Epoch: 17/30...  Training Step: 1035...  Training loss: 4.6923...  1.5150 sec/batch
Epoch: 17/30...  Training Step: 1036...  Training loss: 4.6911...  1.4170 sec/batch
Epoch: 17/30...  Training Step: 1037...  Training loss: 4.6980...  1.3762 sec/batch
Epoch: 17/30...  Training Step: 1038...  Training loss: 4.7190...  1.3719 sec/batch
Epoch: 17/30...  Training Step: 1039...  Training loss: 4.6844...  1.3883 sec/batch
Epoch: 17/30...  Training Step: 1040...  Training loss: 4.7195...  1.3062 sec/batch
Epoch: 17/30...  Training Step: 1041...  Training loss: 4.6467...  1.4753 sec/batch
Epoch: 17/30...  Training Step: 1042...  Training loss: 4.6947...  1.2920 sec/batch
Epoch: 17/30...  Training Step: 1043...  Training loss: 4.6713...  1.3023 sec/batch
Epoch: 17/30...  Training Step: 1044...  Training loss: 4.7076...  1.3523 sec/batch
Epoch: 17/30...  Training Step: 1045...  Training loss: 4.6937...  1.3283 sec/batch
Epoch: 17/30...  Training Step: 1046...  Training loss: 4.6752...  1.2971 sec/batch
Epoch: 17/30...  Training Step: 1047...  Training loss: 4.7030...  1.3797 sec/batch
Epoch: 17/30...  Training Step: 1048...  Training loss: 4.7019...  1.2913 sec/batch
Epoch: 17/30...  Training Step: 1049...  Training loss: 4.6809...  1.3791 sec/batch
Epoch: 17/30...  Training Step: 1050...  Training loss: 4.6829...  1.3875 sec/batch
Epoch: 17/30...  Training Step: 1051...  Training loss: 4.6784...  1.4738 sec/batch
Epoch: 17/30...  Training Step: 1052...  Training loss: 4.6781...  1.2976 sec/batch
Epoch: 17/30...  Training Step: 1053...  Training loss: 4.6645...  1.3171 sec/batch
Epoch: 17/30...  Training Step: 1054...  Training loss: 4.6327...  1.4392 sec/batch
Epoch: 18/30...  Training Step: 1055...  Training loss: 4.7526...  1.4497 sec/batch
Epoch: 18/30...  Training Step: 1056...  Training loss: 4.6383...  1.4230 sec/batch
Epoch: 18/30...  Training Step: 1057...  Training loss: 4.6562...  1.3947 sec/batch
Epoch: 18/30...  Training Step: 1058...  Training loss: 4.7124...  1.3394 sec/batch
Epoch: 18/30...  Training Step: 1059...  Training loss: 4.6570...  1.3759 sec/batch
Epoch: 18/30...  Training Step: 1060...  Training loss: 4.6431...  1.3212 sec/batch
Epoch: 18/30...  Training Step: 1061...  Training loss: 4.6417...  1.4187 sec/batch
Epoch: 18/30...  Training Step: 1062...  Training loss: 4.6741...  1.4355 sec/batch
Epoch: 18/30...  Training Step: 1063...  Training loss: 4.6535...  1.2470 sec/batch
Epoch: 18/30...  Training Step: 1064...  Training loss: 4.6566...  1.3530 sec/batch
Epoch: 18/30...  Training Step: 1065...  Training loss: 4.6264...  1.4564 sec/batch
Epoch: 18/30...  Training Step: 1066...  Training loss: 4.6423...  1.4912 sec/batch
Epoch: 18/30...  Training Step: 1067...  Training loss: 4.6717...  1.4887 sec/batch
Epoch: 18/30...  Training Step: 1068...  Training loss: 4.6443...  1.5669 sec/batch
Epoch: 18/30...  Training Step: 1069...  Training loss: 4.6459...  1.4560 sec/batch
Epoch: 18/30...  Training Step: 1070...  Training loss: 4.6346...  1.3920 sec/batch
Epoch: 18/30...  Training Step: 1071...  Training loss: 4.6548...  1.4066 sec/batch
Epoch: 18/30...  Training Step: 1072...  Training loss: 4.6296...  1.3248 sec/batch
Epoch: 18/30...  Training Step: 1073...  Training loss: 4.5922...  1.2991 sec/batch
Epoch: 18/30...  Training Step: 1074...  Training loss: 4.6366...  1.2811 sec/batch
Epoch: 18/30...  Training Step: 1075...  Training loss: 4.6104...  1.3039 sec/batch
Epoch: 18/30...  Training Step: 1076...  Training loss: 4.6498...  1.4108 sec/batch
Epoch: 18/30...  Training Step: 1077...  Training loss: 4.6543...  1.4577 sec/batch
Epoch: 18/30...  Training Step: 1078...  Training loss: 4.6807...  1.4513 sec/batch
Epoch: 18/30...  Training Step: 1079...  Training loss: 4.6809...  1.3984 sec/batch
Epoch: 18/30...  Training Step: 1080...  Training loss: 4.6997...  1.2941 sec/batch
Epoch: 18/30...  Training Step: 1081...  Training loss: 4.6509...  1.3493 sec/batch
Epoch: 18/30...  Training Step: 1082...  Training loss: 4.6669...  1.3779 sec/batch
Epoch: 18/30...  Training Step: 1083...  Training loss: 4.6524...  1.4170 sec/batch
Epoch: 18/30...  Training Step: 1084...  Training loss: 4.6438...  1.3465 sec/batch
Epoch: 18/30...  Training Step: 1085...  Training loss: 4.6121...  1.3830 sec/batch
Epoch: 18/30...  Training Step: 1086...  Training loss: 4.6029...  1.3931 sec/batch
Epoch: 18/30...  Training Step: 1087...  Training loss: 4.5816...  1.3994 sec/batch
Epoch: 18/30...  Training Step: 1088...  Training loss: 4.6055...  1.3962 sec/batch
Epoch: 18/30...  Training Step: 1089...  Training loss: 4.5776...  1.3627 sec/batch
Epoch: 18/30...  Training Step: 1090...  Training loss: 4.5887...  1.2890 sec/batch
Epoch: 18/30...  Training Step: 1091...  Training loss: 4.6189...  1.4394 sec/batch
Epoch: 18/30...  Training Step: 1092...  Training loss: 4.6200...  1.4582 sec/batch
Epoch: 18/30...  Training Step: 1093...  Training loss: 4.6225...  1.6248 sec/batch
Epoch: 18/30...  Training Step: 1094...  Training loss: 4.6469...  1.3910 sec/batch
Epoch: 18/30...  Training Step: 1095...  Training loss: 4.6084...  1.7161 sec/batch
Epoch: 18/30...  Training Step: 1096...  Training loss: 4.6232...  1.4242 sec/batch
Epoch: 18/30...  Training Step: 1097...  Training loss: 4.5986...  1.3628 sec/batch
Epoch: 18/30...  Training Step: 1098...  Training loss: 4.6020...  1.4116 sec/batch
Epoch: 18/30...  Training Step: 1099...  Training loss: 4.5915...  1.4350 sec/batch
Epoch: 18/30...  Training Step: 1100...  Training loss: 4.6110...  1.3086 sec/batch
Epoch: 18/30...  Training Step: 1101...  Training loss: 4.5906...  1.4335 sec/batch
Epoch: 18/30...  Training Step: 1102...  Training loss: 4.6226...  1.4950 sec/batch
Epoch: 18/30...  Training Step: 1103...  Training loss: 4.5560...  1.5523 sec/batch
Epoch: 18/30...  Training Step: 1104...  Training loss: 4.6098...  1.4550 sec/batch
Epoch: 18/30...  Training Step: 1105...  Training loss: 4.5810...  1.3357 sec/batch
Epoch: 18/30...  Training Step: 1106...  Training loss: 4.6141...  1.4168 sec/batch
Epoch: 18/30...  Training Step: 1107...  Training loss: 4.6087...  1.3556 sec/batch
Epoch: 18/30...  Training Step: 1108...  Training loss: 4.5898...  1.3724 sec/batch
Epoch: 18/30...  Training Step: 1109...  Training loss: 4.6050...  1.4416 sec/batch
Epoch: 18/30...  Training Step: 1110...  Training loss: 4.6102...  1.4068 sec/batch
Epoch: 18/30...  Training Step: 1111...  Training loss: 4.5848...  1.3328 sec/batch
Epoch: 18/30...  Training Step: 1112...  Training loss: 4.5751...  1.3418 sec/batch
Epoch: 18/30...  Training Step: 1113...  Training loss: 4.5814...  1.4757 sec/batch
Epoch: 18/30...  Training Step: 1114...  Training loss: 4.5737...  1.3808 sec/batch
Epoch: 18/30...  Training Step: 1115...  Training loss: 4.5599...  1.3489 sec/batch
Epoch: 18/30...  Training Step: 1116...  Training loss: 4.5335...  1.4536 sec/batch
Epoch: 19/30...  Training Step: 1117...  Training loss: 4.6411...  1.4858 sec/batch
Epoch: 19/30...  Training Step: 1118...  Training loss: 4.5314...  1.4491 sec/batch
Epoch: 19/30...  Training Step: 1119...  Training loss: 4.5438...  1.2914 sec/batch
Epoch: 19/30...  Training Step: 1120...  Training loss: 4.6099...  1.3499 sec/batch
Epoch: 19/30...  Training Step: 1121...  Training loss: 4.5529...  1.2935 sec/batch
Epoch: 19/30...  Training Step: 1122...  Training loss: 4.5485...  1.4095 sec/batch
Epoch: 19/30...  Training Step: 1123...  Training loss: 4.5463...  1.3023 sec/batch
Epoch: 19/30...  Training Step: 1124...  Training loss: 4.5749...  1.4587 sec/batch
Epoch: 19/30...  Training Step: 1125...  Training loss: 4.5477...  1.5017 sec/batch
Epoch: 19/30...  Training Step: 1126...  Training loss: 4.5687...  1.4345 sec/batch
Epoch: 19/30...  Training Step: 1127...  Training loss: 4.5231...  1.3855 sec/batch
Epoch: 19/30...  Training Step: 1128...  Training loss: 4.5388...  1.3006 sec/batch
Epoch: 19/30...  Training Step: 1129...  Training loss: 4.5660...  1.3189 sec/batch
Epoch: 19/30...  Training Step: 1130...  Training loss: 4.5456...  1.3899 sec/batch
Epoch: 19/30...  Training Step: 1131...  Training loss: 4.5532...  1.4822 sec/batch
Epoch: 19/30...  Training Step: 1132...  Training loss: 4.5316...  1.4806 sec/batch
Epoch: 19/30...  Training Step: 1133...  Training loss: 4.5533...  1.5225 sec/batch
Epoch: 19/30...  Training Step: 1134...  Training loss: 4.5407...  1.5285 sec/batch
Epoch: 19/30...  Training Step: 1135...  Training loss: 4.4997...  1.4936 sec/batch
Epoch: 19/30...  Training Step: 1136...  Training loss: 4.5536...  1.5063 sec/batch
Epoch: 19/30...  Training Step: 1137...  Training loss: 4.5134...  1.6194 sec/batch
Epoch: 19/30...  Training Step: 1138...  Training loss: 4.5499...  1.6675 sec/batch
Epoch: 19/30...  Training Step: 1139...  Training loss: 4.5475...  1.5627 sec/batch
Epoch: 19/30...  Training Step: 1140...  Training loss: 4.5825...  1.3919 sec/batch
Epoch: 19/30...  Training Step: 1141...  Training loss: 4.5898...  1.3486 sec/batch
Epoch: 19/30...  Training Step: 1142...  Training loss: 4.5879...  1.3324 sec/batch
Epoch: 19/30...  Training Step: 1143...  Training loss: 4.5470...  1.7314 sec/batch
Epoch: 19/30...  Training Step: 1144...  Training loss: 4.5853...  1.6944 sec/batch
Epoch: 19/30...  Training Step: 1145...  Training loss: 4.5598...  1.4480 sec/batch
Epoch: 19/30...  Training Step: 1146...  Training loss: 4.5436...  1.4264 sec/batch
Epoch: 19/30...  Training Step: 1147...  Training loss: 4.5054...  1.4672 sec/batch
Epoch: 19/30...  Training Step: 1148...  Training loss: 4.5110...  1.3892 sec/batch
Epoch: 19/30...  Training Step: 1149...  Training loss: 4.4986...  1.3184 sec/batch
Epoch: 19/30...  Training Step: 1150...  Training loss: 4.4971...  1.3138 sec/batch
Epoch: 19/30...  Training Step: 1151...  Training loss: 4.4809...  1.4541 sec/batch
Epoch: 19/30...  Training Step: 1152...  Training loss: 4.4978...  1.5291 sec/batch
Epoch: 19/30...  Training Step: 1153...  Training loss: 4.5208...  1.3467 sec/batch
Epoch: 19/30...  Training Step: 1154...  Training loss: 4.5352...  1.4835 sec/batch
Epoch: 19/30...  Training Step: 1155...  Training loss: 4.5418...  1.4051 sec/batch
Epoch: 19/30...  Training Step: 1156...  Training loss: 4.5546...  1.4698 sec/batch
Epoch: 19/30...  Training Step: 1157...  Training loss: 4.5280...  1.5614 sec/batch
Epoch: 19/30...  Training Step: 1158...  Training loss: 4.5361...  1.5961 sec/batch
Epoch: 19/30...  Training Step: 1159...  Training loss: 4.5165...  1.5009 sec/batch
Epoch: 19/30...  Training Step: 1160...  Training loss: 4.5028...  1.4800 sec/batch
Epoch: 19/30...  Training Step: 1161...  Training loss: 4.5096...  1.5097 sec/batch
Epoch: 19/30...  Training Step: 1162...  Training loss: 4.5282...  1.3100 sec/batch
Epoch: 19/30...  Training Step: 1163...  Training loss: 4.5003...  1.3816 sec/batch
Epoch: 19/30...  Training Step: 1164...  Training loss: 4.5268...  1.4187 sec/batch
Epoch: 19/30...  Training Step: 1165...  Training loss: 4.4596...  1.4622 sec/batch
Epoch: 19/30...  Training Step: 1166...  Training loss: 4.5200...  1.4429 sec/batch
Epoch: 19/30...  Training Step: 1167...  Training loss: 4.4879...  1.4332 sec/batch
Epoch: 19/30...  Training Step: 1168...  Training loss: 4.5222...  1.4289 sec/batch
Epoch: 19/30...  Training Step: 1169...  Training loss: 4.5163...  1.4635 sec/batch
Epoch: 19/30...  Training Step: 1170...  Training loss: 4.4966...  1.3960 sec/batch
Epoch: 19/30...  Training Step: 1171...  Training loss: 4.5004...  1.3610 sec/batch
Epoch: 19/30...  Training Step: 1172...  Training loss: 4.5168...  1.4984 sec/batch
Epoch: 19/30...  Training Step: 1173...  Training loss: 4.4985...  1.4341 sec/batch
Epoch: 19/30...  Training Step: 1174...  Training loss: 4.4861...  1.4496 sec/batch
Epoch: 19/30...  Training Step: 1175...  Training loss: 4.4836...  1.4010 sec/batch
Epoch: 19/30...  Training Step: 1176...  Training loss: 4.4897...  1.3587 sec/batch
Epoch: 19/30...  Training Step: 1177...  Training loss: 4.4745...  1.3382 sec/batch
Epoch: 19/30...  Training Step: 1178...  Training loss: 4.4456...  1.3796 sec/batch
Epoch: 20/30...  Training Step: 1179...  Training loss: 4.5529...  1.5468 sec/batch
Epoch: 20/30...  Training Step: 1180...  Training loss: 4.4555...  1.4445 sec/batch
Epoch: 20/30...  Training Step: 1181...  Training loss: 4.4687...  1.4066 sec/batch
Epoch: 20/30...  Training Step: 1182...  Training loss: 4.5350...  1.4974 sec/batch
Epoch: 20/30...  Training Step: 1183...  Training loss: 4.4910...  1.2922 sec/batch
Epoch: 20/30...  Training Step: 1184...  Training loss: 4.4894...  1.3800 sec/batch
Epoch: 20/30...  Training Step: 1185...  Training loss: 4.4749...  1.3368 sec/batch
Epoch: 20/30...  Training Step: 1186...  Training loss: 4.5101...  1.3806 sec/batch
Epoch: 20/30...  Training Step: 1187...  Training loss: 4.4920...  1.3216 sec/batch
Epoch: 20/30...  Training Step: 1188...  Training loss: 4.4925...  1.4259 sec/batch
Epoch: 20/30...  Training Step: 1189...  Training loss: 4.4682...  1.3474 sec/batch
Epoch: 20/30...  Training Step: 1190...  Training loss: 4.4801...  1.3819 sec/batch
Epoch: 20/30...  Training Step: 1191...  Training loss: 4.4990...  1.2423 sec/batch
Epoch: 20/30...  Training Step: 1192...  Training loss: 4.4649...  1.4866 sec/batch
Epoch: 20/30...  Training Step: 1193...  Training loss: 4.4688...  1.3511 sec/batch
Epoch: 20/30...  Training Step: 1194...  Training loss: 4.4665...  1.3977 sec/batch
Epoch: 20/30...  Training Step: 1195...  Training loss: 4.4857...  1.3426 sec/batch
Epoch: 20/30...  Training Step: 1196...  Training loss: 4.4597...  1.4367 sec/batch
Epoch: 20/30...  Training Step: 1197...  Training loss: 4.4293...  1.4250 sec/batch
Epoch: 20/30...  Training Step: 1198...  Training loss: 4.4726...  1.3111 sec/batch
Epoch: 20/30...  Training Step: 1199...  Training loss: 4.4360...  1.5133 sec/batch
Epoch: 20/30...  Training Step: 1200...  Training loss: 4.4886...  1.4008 sec/batch
Epoch: 20/30...  Training Step: 1201...  Training loss: 4.4752...  1.4025 sec/batch
Epoch: 20/30...  Training Step: 1202...  Training loss: 4.4974...  1.4832 sec/batch
Epoch: 20/30...  Training Step: 1203...  Training loss: 4.5042...  1.5814 sec/batch
Epoch: 20/30...  Training Step: 1204...  Training loss: 4.5098...  1.5805 sec/batch
Epoch: 20/30...  Training Step: 1205...  Training loss: 4.4724...  1.4611 sec/batch
Epoch: 20/30...  Training Step: 1206...  Training loss: 4.5090...  1.4367 sec/batch
Epoch: 20/30...  Training Step: 1207...  Training loss: 4.4799...  1.5450 sec/batch
Epoch: 20/30...  Training Step: 1208...  Training loss: 4.4712...  1.4582 sec/batch
Epoch: 20/30...  Training Step: 1209...  Training loss: 4.4403...  1.4702 sec/batch
Epoch: 20/30...  Training Step: 1210...  Training loss: 4.4435...  1.4807 sec/batch
Epoch: 20/30...  Training Step: 1211...  Training loss: 4.4199...  1.6853 sec/batch
Epoch: 20/30...  Training Step: 1212...  Training loss: 4.4314...  1.4890 sec/batch
Epoch: 20/30...  Training Step: 1213...  Training loss: 4.4177...  1.5145 sec/batch
Epoch: 20/30...  Training Step: 1214...  Training loss: 4.4266...  1.5016 sec/batch
Epoch: 20/30...  Training Step: 1215...  Training loss: 4.4521...  1.3838 sec/batch
Epoch: 20/30...  Training Step: 1216...  Training loss: 4.4597...  1.3815 sec/batch
Epoch: 20/30...  Training Step: 1217...  Training loss: 4.4678...  1.3587 sec/batch
Epoch: 20/30...  Training Step: 1218...  Training loss: 4.4846...  1.4757 sec/batch
Epoch: 20/30...  Training Step: 1219...  Training loss: 4.4434...  1.3514 sec/batch
Epoch: 20/30...  Training Step: 1220...  Training loss: 4.4653...  1.3911 sec/batch
Epoch: 20/30...  Training Step: 1221...  Training loss: 4.4490...  1.3059 sec/batch
Epoch: 20/30...  Training Step: 1222...  Training loss: 4.4405...  1.3211 sec/batch
Epoch: 20/30...  Training Step: 1223...  Training loss: 4.4390...  1.5048 sec/batch
Epoch: 20/30...  Training Step: 1224...  Training loss: 4.4594...  1.3678 sec/batch
Epoch: 20/30...  Training Step: 1225...  Training loss: 4.4203...  1.3078 sec/batch
Epoch: 20/30...  Training Step: 1226...  Training loss: 4.4573...  1.3658 sec/batch
Epoch: 20/30...  Training Step: 1227...  Training loss: 4.3906...  1.3634 sec/batch
Epoch: 20/30...  Training Step: 1228...  Training loss: 4.4475...  1.3819 sec/batch
Epoch: 20/30...  Training Step: 1229...  Training loss: 4.4118...  1.3966 sec/batch
Epoch: 20/30...  Training Step: 1230...  Training loss: 4.4667...  1.3597 sec/batch
Epoch: 20/30...  Training Step: 1231...  Training loss: 4.4639...  1.4550 sec/batch
Epoch: 20/30...  Training Step: 1232...  Training loss: 4.4355...  1.4259 sec/batch
Epoch: 20/30...  Training Step: 1233...  Training loss: 4.4346...  1.4823 sec/batch
Epoch: 20/30...  Training Step: 1234...  Training loss: 4.4463...  1.4522 sec/batch
Epoch: 20/30...  Training Step: 1235...  Training loss: 4.4354...  1.3926 sec/batch
Epoch: 20/30...  Training Step: 1236...  Training loss: 4.4262...  1.2481 sec/batch
Epoch: 20/30...  Training Step: 1237...  Training loss: 4.4246...  1.4239 sec/batch
Epoch: 20/30...  Training Step: 1238...  Training loss: 4.4316...  1.4931 sec/batch
Epoch: 20/30...  Training Step: 1239...  Training loss: 4.4041...  1.3734 sec/batch
Epoch: 20/30...  Training Step: 1240...  Training loss: 4.3765...  1.4841 sec/batch
Epoch: 21/30...  Training Step: 1241...  Training loss: 4.4840...  1.5626 sec/batch
Epoch: 21/30...  Training Step: 1242...  Training loss: 4.3858...  1.3928 sec/batch
Epoch: 21/30...  Training Step: 1243...  Training loss: 4.3998...  1.4124 sec/batch
Epoch: 21/30...  Training Step: 1244...  Training loss: 4.4483...  1.5940 sec/batch
Epoch: 21/30...  Training Step: 1245...  Training loss: 4.4034...  1.3914 sec/batch
Epoch: 21/30...  Training Step: 1246...  Training loss: 4.4117...  1.3731 sec/batch
Epoch: 21/30...  Training Step: 1247...  Training loss: 4.3987...  1.4159 sec/batch
Epoch: 21/30...  Training Step: 1248...  Training loss: 4.4151...  1.4279 sec/batch
Epoch: 21/30...  Training Step: 1249...  Training loss: 4.4087...  1.2862 sec/batch
Epoch: 21/30...  Training Step: 1250...  Training loss: 4.4263...  1.3412 sec/batch
Epoch: 21/30...  Training Step: 1251...  Training loss: 4.3757...  1.3069 sec/batch
Epoch: 21/30...  Training Step: 1252...  Training loss: 4.3795...  1.3149 sec/batch
Epoch: 21/30...  Training Step: 1253...  Training loss: 4.4253...  1.4985 sec/batch
Epoch: 21/30...  Training Step: 1254...  Training loss: 4.3915...  1.3205 sec/batch
Epoch: 21/30...  Training Step: 1255...  Training loss: 4.4043...  1.3300 sec/batch
Epoch: 21/30...  Training Step: 1256...  Training loss: 4.3960...  1.6422 sec/batch
Epoch: 21/30...  Training Step: 1257...  Training loss: 4.4025...  1.3964 sec/batch
Epoch: 21/30...  Training Step: 1258...  Training loss: 4.4060...  1.3395 sec/batch
Epoch: 21/30...  Training Step: 1259...  Training loss: 4.3611...  1.6280 sec/batch
Epoch: 21/30...  Training Step: 1260...  Training loss: 4.4081...  1.5186 sec/batch
Epoch: 21/30...  Training Step: 1261...  Training loss: 4.3647...  1.3686 sec/batch
Epoch: 21/30...  Training Step: 1262...  Training loss: 4.4174...  1.7534 sec/batch
Epoch: 21/30...  Training Step: 1263...  Training loss: 4.3995...  1.3968 sec/batch
Epoch: 21/30...  Training Step: 1264...  Training loss: 4.4152...  1.3251 sec/batch
Epoch: 21/30...  Training Step: 1265...  Training loss: 4.4437...  1.3935 sec/batch
Epoch: 21/30...  Training Step: 1266...  Training loss: 4.4324...  1.4310 sec/batch
Epoch: 21/30...  Training Step: 1267...  Training loss: 4.4060...  1.3113 sec/batch
Epoch: 21/30...  Training Step: 1268...  Training loss: 4.4243...  1.4595 sec/batch
Epoch: 21/30...  Training Step: 1269...  Training loss: 4.3907...  1.5401 sec/batch
Epoch: 21/30...  Training Step: 1270...  Training loss: 4.4000...  1.3892 sec/batch
Epoch: 21/30...  Training Step: 1271...  Training loss: 4.3650...  1.3141 sec/batch
Epoch: 21/30...  Training Step: 1272...  Training loss: 4.3769...  1.4278 sec/batch
Epoch: 21/30...  Training Step: 1273...  Training loss: 4.3741...  1.4520 sec/batch
Epoch: 21/30...  Training Step: 1274...  Training loss: 4.3673...  1.3736 sec/batch
Epoch: 21/30...  Training Step: 1275...  Training loss: 4.3462...  1.2347 sec/batch
Epoch: 21/30...  Training Step: 1276...  Training loss: 4.3678...  1.3759 sec/batch
Epoch: 21/30...  Training Step: 1277...  Training loss: 4.3851...  1.3915 sec/batch
Epoch: 21/30...  Training Step: 1278...  Training loss: 4.3875...  1.4297 sec/batch
Epoch: 21/30...  Training Step: 1279...  Training loss: 4.3907...  1.3359 sec/batch
Epoch: 21/30...  Training Step: 1280...  Training loss: 4.4133...  1.4500 sec/batch
Epoch: 21/30...  Training Step: 1281...  Training loss: 4.3735...  1.4441 sec/batch
Epoch: 21/30...  Training Step: 1282...  Training loss: 4.3945...  1.6365 sec/batch
Epoch: 21/30...  Training Step: 1283...  Training loss: 4.3757...  1.3608 sec/batch
Epoch: 21/30...  Training Step: 1284...  Training loss: 4.3734...  1.5281 sec/batch
Epoch: 21/30...  Training Step: 1285...  Training loss: 4.3781...  1.2848 sec/batch
Epoch: 21/30...  Training Step: 1286...  Training loss: 4.3788...  1.2916 sec/batch
Epoch: 21/30...  Training Step: 1287...  Training loss: 4.3612...  1.3917 sec/batch
Epoch: 21/30...  Training Step: 1288...  Training loss: 4.3836...  1.4029 sec/batch
Epoch: 21/30...  Training Step: 1289...  Training loss: 4.3124...  1.3698 sec/batch
Epoch: 21/30...  Training Step: 1290...  Training loss: 4.3811...  1.3111 sec/batch
Epoch: 21/30...  Training Step: 1291...  Training loss: 4.3527...  1.3352 sec/batch
Epoch: 21/30...  Training Step: 1292...  Training loss: 4.3879...  1.3829 sec/batch
Epoch: 21/30...  Training Step: 1293...  Training loss: 4.3709...  1.3836 sec/batch
Epoch: 21/30...  Training Step: 1294...  Training loss: 4.3539...  1.3935 sec/batch
Epoch: 21/30...  Training Step: 1295...  Training loss: 4.3597...  1.4011 sec/batch
Epoch: 21/30...  Training Step: 1296...  Training loss: 4.3890...  1.5598 sec/batch
Epoch: 21/30...  Training Step: 1297...  Training loss: 4.3587...  1.3401 sec/batch
Epoch: 21/30...  Training Step: 1298...  Training loss: 4.3537...  1.3893 sec/batch
Epoch: 21/30...  Training Step: 1299...  Training loss: 4.3620...  1.6613 sec/batch
Epoch: 21/30...  Training Step: 1300...  Training loss: 4.3662...  1.3668 sec/batch
Epoch: 21/30...  Training Step: 1301...  Training loss: 4.3330...  1.5121 sec/batch
Epoch: 21/30...  Training Step: 1302...  Training loss: 4.3027...  1.5596 sec/batch
Epoch: 22/30...  Training Step: 1303...  Training loss: 4.4031...  1.5044 sec/batch
Epoch: 22/30...  Training Step: 1304...  Training loss: 4.3100...  1.3963 sec/batch
Epoch: 22/30...  Training Step: 1305...  Training loss: 4.3301...  1.4015 sec/batch
Epoch: 22/30...  Training Step: 1306...  Training loss: 4.3828...  1.3982 sec/batch
Epoch: 22/30...  Training Step: 1307...  Training loss: 4.3413...  1.3851 sec/batch
Epoch: 22/30...  Training Step: 1308...  Training loss: 4.3266...  1.3596 sec/batch
Epoch: 22/30...  Training Step: 1309...  Training loss: 4.3202...  1.3936 sec/batch
Epoch: 22/30...  Training Step: 1310...  Training loss: 4.3480...  1.3037 sec/batch
Epoch: 22/30...  Training Step: 1311...  Training loss: 4.3297...  1.3263 sec/batch
Epoch: 22/30...  Training Step: 1312...  Training loss: 4.3397...  1.5033 sec/batch
Epoch: 22/30...  Training Step: 1313...  Training loss: 4.3156...  1.5217 sec/batch
Epoch: 22/30...  Training Step: 1314...  Training loss: 4.3240...  1.4009 sec/batch
Epoch: 22/30...  Training Step: 1315...  Training loss: 4.3369...  1.3788 sec/batch
Epoch: 22/30...  Training Step: 1316...  Training loss: 4.3261...  1.2972 sec/batch
Epoch: 22/30...  Training Step: 1317...  Training loss: 4.3299...  1.3601 sec/batch
Epoch: 22/30...  Training Step: 1318...  Training loss: 4.3154...  1.3973 sec/batch
Epoch: 22/30...  Training Step: 1319...  Training loss: 4.3337...  1.6458 sec/batch
Epoch: 22/30...  Training Step: 1320...  Training loss: 4.3254...  1.3115 sec/batch
Epoch: 22/30...  Training Step: 1321...  Training loss: 4.2980...  1.4674 sec/batch
Epoch: 22/30...  Training Step: 1322...  Training loss: 4.3379...  1.5256 sec/batch
Epoch: 22/30...  Training Step: 1323...  Training loss: 4.2973...  1.5510 sec/batch
Epoch: 22/30...  Training Step: 1324...  Training loss: 4.3434...  1.4782 sec/batch
Epoch: 22/30...  Training Step: 1325...  Training loss: 4.3538...  1.4280 sec/batch
Epoch: 22/30...  Training Step: 1326...  Training loss: 4.3684...  1.3242 sec/batch
Epoch: 22/30...  Training Step: 1327...  Training loss: 4.3619...  1.4754 sec/batch
Epoch: 22/30...  Training Step: 1328...  Training loss: 4.3610...  1.3826 sec/batch
Epoch: 22/30...  Training Step: 1329...  Training loss: 4.3262...  1.3355 sec/batch
Epoch: 22/30...  Training Step: 1330...  Training loss: 4.3648...  1.2858 sec/batch
Epoch: 22/30...  Training Step: 1331...  Training loss: 4.3412...  1.3501 sec/batch
Epoch: 22/30...  Training Step: 1332...  Training loss: 4.3185...  1.4523 sec/batch
Epoch: 22/30...  Training Step: 1333...  Training loss: 4.3029...  1.4473 sec/batch
Epoch: 22/30...  Training Step: 1334...  Training loss: 4.3034...  1.4883 sec/batch
Epoch: 22/30...  Training Step: 1335...  Training loss: 4.2962...  1.3826 sec/batch
Epoch: 22/30...  Training Step: 1336...  Training loss: 4.2992...  1.3938 sec/batch
Epoch: 22/30...  Training Step: 1337...  Training loss: 4.2842...  1.3821 sec/batch
Epoch: 22/30...  Training Step: 1338...  Training loss: 4.3023...  1.6591 sec/batch
Epoch: 22/30...  Training Step: 1339...  Training loss: 4.3201...  1.3313 sec/batch
Epoch: 22/30...  Training Step: 1340...  Training loss: 4.3328...  1.3389 sec/batch
Epoch: 22/30...  Training Step: 1341...  Training loss: 4.3354...  1.4726 sec/batch
Epoch: 22/30...  Training Step: 1342...  Training loss: 4.3472...  1.3731 sec/batch
Epoch: 22/30...  Training Step: 1343...  Training loss: 4.3096...  1.3560 sec/batch
Epoch: 22/30...  Training Step: 1344...  Training loss: 4.3393...  1.3048 sec/batch
Epoch: 22/30...  Training Step: 1345...  Training loss: 4.3174...  1.2819 sec/batch
Epoch: 22/30...  Training Step: 1346...  Training loss: 4.3188...  1.2545 sec/batch
Epoch: 22/30...  Training Step: 1347...  Training loss: 4.3166...  1.3902 sec/batch
Epoch: 22/30...  Training Step: 1348...  Training loss: 4.3272...  1.2909 sec/batch
Epoch: 22/30...  Training Step: 1349...  Training loss: 4.3107...  1.3237 sec/batch
Epoch: 22/30...  Training Step: 1350...  Training loss: 4.3345...  1.3181 sec/batch

跑22轮就自动终止了，看来2核8G的机子也不太行，，数据量有点大

**Saved checkpoints**

关于保存checkpoints的说明可以看这个 https://www.tensorflow.org/programmers_guide/variables

In [18]:
tf.train.get_checkpoint_state('checkpoints')

**采样**

现在网络已经训练好了，我们可以用它来生成新的文本。这个想法是，我们传入一个字符，然后网络会预测下一个字符。我们可以用新的，来预测下一个。我们继续这样做来生成新的文本。我还包含了一些功能，通过传递一个字符串并从中构建一个状态，从而为网络提供一些文本。



网络为我们提供每个字符的预测。为了减少噪音，减少一些随机性，我将只从前N个最有可能的字符中选择一个新字符。



In [19]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [20]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

在这里，传递到检查点的路径并从网络中取样。

In [21]:
tf.train.latest_checkpoint('checkpoints')

**五、预测结果**

这里我选取了五句诗句来测试下训练好的模型，感觉效果还是不错的（不是自夸，看对出来的诗句感觉也是有一定水平滴）

总体来说，训练的模型达到了预期的效果

In [2]:
sent = ' '.join(SingCut('一干二净除旧习'))
auto_couplet(sent, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

上联: <start> 一 干 二 净 除 旧 习 <end>
下联: <start> 三 分 一 路 寻 常 态 <end> 


In [6]:
sent = ' '.join(SingCut('日出江花红胜火'))
auto_couplet(sent, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

上联: <start> 日 出 江 花 红 胜 火 <end>
下联: <start> 风 吹 月 色 春 秋 色 <end>


In [8]:
sent = ' '.join(SingCut('壮丽山河多异彩'))
auto_couplet(sent, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

上联: <start> 壮 丽 山 河 多 异 彩 <end>
下联: <start> 清 明 月 色 清 风 劲 <end> 


In [9]:
sent = ' '.join(SingCut('新千年迎新春'))
auto_couplet(sent, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

上联: <start> 新 千 年 迎 新 春 <end>
下联: <start> 大 小 康 庄 严 月 <end>  


In [10]:
sent = ' '.join(SingCut('我欲乘风归去'))
auto_couplet(sent, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

上联: <start> 我 欲 乘 风 归 去 <end>
下联: <start> 谁 能 解 语 知 浅 <end>


​ **六、番外篇—项目衍生出的实际应用&项目之外的工作->打个广告**

**番外**

单纯字符的输出不太好看，我们可以利用网上的公开字模库，包装我们的对子，主要步骤就是导入相应的库，下载字模、背景图，生成详细看代码吧

In [ ]:
import io
from PIL import Image
import numpy as np
import requests
def get_word(ch, quality):
    """获取单个汉字（字符）的图片
    ch          - 单个汉字或英文字母（仅支持大写）
    quality     - 单字分辨率，H-640像素，M-480像素，L-320像素
    """
    
    fp = io.BytesIO(requests.post(url='http://xufive.sdysit.com/tk', data={'ch':ch}).content)
    im = Image.open(fp)
    w, h = im.size
    if quality == 'M':
        w, h = int(w*0.75), int(0.75*h)
    elif quality == 'L':
        w, h = int(w*0.5), int(0.5*h)
    
    return im.resize((w,h))
def get_bg(quality):
    """获取对子背景的图片"""
    
    return get_word('bg', quality)
def write_couplets(text, HorV='V', quality='L', out_file=None):
    """生成对子
    
    text        - 春联内容，以空格断行
    HorV        - H-横排，V-竖排
    quality     - 单字分辨率，H-640像素，M-480像素，L-320像素
    out_file    - 输出文件名
    """
    
    usize = {'H':(640,23), 'M':(480,18), 'L':(320,12)}
    bg_im = get_bg(quality)
    text_list = [list(item) for item in text.split()]
    rows = len(text_list)
    cols = max([len(item) for item in text_list])
    
    if HorV == 'V':
        ow, oh = 40+rows*usize[quality][0]+(rows-1)*10, 40+cols*usize[quality][0]
    else:
        ow, oh = 40+cols*usize[quality][0], 40+rows*usize[quality][0]+(rows-1)*10
    out_im = Image.new('RGBA', (ow, oh), '#f0f0f0')
    
    for row in range(rows):
        if HorV == 'V':
            row_im = Image.new('RGBA', (usize[quality][0], cols*usize[quality][0]), 'white')
            offset = (ow-(usize[quality][0]+10)*(row+1)-10, 20)
        else:
            row_im = Image.new('RGBA', (cols*usize[quality][0], usize[quality][0]), 'white')
            offset = (20, 20+(usize[quality][0]+10)*row)
        
        for col, ch in enumerate(text_list[row]):
            if HorV == 'V':
                pos = (0, col*usize[quality][0])
            else:
                pos = (col*usize[quality][0],0)
            
            ch_im = get_word(ch, quality)
            row_im.paste(bg_im, pos)
            row_im.paste(ch_im, (pos[0]+usize[quality][1], pos[1]+usize[quality][1]), mask=ch_im)
            
        out_im.paste(row_im, offset)
    
    if out_file:
        out_im.convert('RGB').save(out_file)
    out_im.show()
text = output
write_couplets(text, HorV='V', quality='M', out_file='final.jpg')




由于篇幅受限，这里就展示一部分，生成的对联，借用上面的输出结果：
[![tzdZf1.md.jpg](https://s1.ax1x.com/2020/06/14/tzdZf1.md.jpg)](https://imgchr.com/i/tzdZf1)
[![tzdmSx.md.jpg](https://s1.ax1x.com/2020/06/14/tzdmSx.md.jpg)](https://imgchr.com/i/tzdmSx)
[![tzdnl6.md.jpg](https://s1.ax1x.com/2020/06/14/tzdnl6.md.jpg)](https://imgchr.com/i/tzdnl6)

至此，项目大致展示完毕

**关于项目之外的，emm其实也没什么好说的，只是在4月末5月初的时候得知了期末要做一个有关机器学习的大项目的事情那时候就在着手做一个有关数据分析的东西了，但是后来做完之后发现不太对题就没有在做了，要是有兴趣的话可以看我的代码部分的另外一个项目**

最后，感谢王程老师，感谢吴德文老师，感谢陈龙彪老师，一学期的模式识别学习真的让我学会了很多，算是一个很nice的启蒙吧，谢谢！